# Advent of Code 2021 - APL

[Advent of Code](https://adventofcode.com/) is a daily programming puzzle
challenge published each December. The puzzles are pretty easy for experienced
programmers, so they're a great way to play around with new programming
languages and tools. This year I'm trying out
[APL](https://en.wikipedia.org/wiki/APL_(programming_language)), a famously
terse array language from the 60s. You may have seen its memeable program for
[Conway's game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life):

In [33]:
gol ← {≢⍸⍵}⌺3 3∊¨3+0,¨⊢ ⍝ Generate the next generation 🤯

A few people expressed interest, so about halfway through December I decided
to try publishing a Jupyter notebook with my solutions in literate programming
fashion. Caveat that I'm a complete APL newb so some of my answers may be
really bad/ugly! But I'm having fun. After a few weeks, the funny symbols are
starting to make some sense, and I'm contemplating buying a special keyboard...

## Resources

[Dyalog](https://www.dyalog.com/) seems to be the APL that most people use.
Some good online resources are [APLcart](https://aplcart.info/),
[Learning APL](https://xpqz.github.io/learnapl/), and
[Mastering Dyalog APL](https://mastering.dyalog.com/README.html).

A few days in, YouTube's algorithm told me about another crazy person solving
AoC in APL! [code_report](https://www.youtube.com/c/codereport) seems to be much
better at it and for the first few days, I compared notes with their videos
and learned a ton.

## Day 1

[Puzzle link](https://adventofcode.com/2021/day/1)

### Part 1

Today we're just supposed to count increasing numbers in a list, which takes
all of 5 characters.

In [7]:
test1←199 200 208 210 200 207 240 269 260 263

In [13]:
+/2</test1 ⍝ reduce test1 pairwise on < then sum the resulting boolean array

7

Note that APL is read right to left. The parser is really, really basic, and
I keep shooting myself in the foot by forgetting parentheses.

The pinacle of APL style is "tacit" programming involving just "trains" of
functions and operators smushed together with no explicit arguments. We can
store and reuse the solution for day 1, part 1, thusly

In [30]:
day1_1←+/2</⊢ ⍝ the ⊢ means "I'm with stupid"

In [31]:
day1_1 test1

7

Reading stuff from a file took embarrasingly long to figure out.

In [38]:
data1←⍎¨⊃⎕NGET 'day1-input.txt' 1 ⍝ read lines and eval each one turning strings to numbers

In [39]:
day1_1 data1

1390

### Part 2

Instead we need to sum every three adjacent elements and then count increases,
which takes a whole 8 characters.

In [34]:
+/2</3+/test1 ⍝ reduce test1 on 3-wise sums, pairwise compare, and sum increases

5

In [35]:
day1_2←+/2</3+/⊢

In [41]:
day1_2 data1

1457

## Day 2

[Puzzle link](https://adventofcode.com/2021/day/2)

Today we have to deal with words to drive a submarine around. APL strings are
(predictably) just arrays, and there don't seem to be special string libraries.
So let's turn this all into numbers to simplify life.

We'll use a "dfn" i.e. a function to parse lines. The dfn grammar is powerful
enough that once you understand it, you can basically use APL as a terrible
functional language.

In [94]:
]dinput
parse_sub_command←{
  ⍝ converts a command like 'forward 5' into two numbers
  dir delta←' '(≠⊆⊢)⍵      ⍝ split on space (i.e. partition on not-equal ' ')
  dir≡'forward':(⍎delta)0  ⍝ 'forward x': x 0
  dir≡'down':0(⍎delta)     ⍝ 'down x': 0 x
  0(-⍎delta)               ⍝ 'up x': 0 -x
}

In [95]:
⎕←test2←parse_sub_command¨⊃⎕NGET 'day2-test.txt' 1 ⍝ map parser over lines

┌───┬───┬───┬────┬───┬───┐
│5 0│0 5│8 0│0 ¯3│0 8│2 0│
└───┴───┴───┴────┴───┴───┘

In [96]:
data2←parse_sub_command¨⊃⎕NGET 'day2-input.txt' 1

### Part 1

First we're supposed to move the sub, then return its position times its
depth to prove we parsed the commands ok. We can just add up our tuples
then multiply the components.

In [105]:
day2_1←×/∘↑(+/⊢) ⍝ sum reduce, unbox, then multiply

In [106]:
day2_1 test2

150

In [107]:
day2_1 data2

1938402

### Part 2

jk actually there's another state variable aim. We can make this variable
explicit using a scan which is a reduce that gives you partial answers.

In [108]:
+\2⊃¨test2 ⍝ + scan over seconds of test2 (i.e. up/down amounts)

0 5 5 2 10 10

So to get depth we multiply aim by the X for forward commands, then sum.
Horizontal pos is just the sum of the firsts as before.

In [110]:
day2_2←{(+/⊃¨⍵)×+/(+\2⊃¨⍵)×(⊃¨⍵)} ⍝ sum aim×X, then multiply horizontal pos

In [111]:
day2_2 test2

900

In [112]:
day2_2 data2

1947878632

If you wanted you could probably trim some characters and write this tacitly
but it's already needlessly obtuse imo, and reads better with some names for
things.

In [117]:
]dinput
day2_2←{
  forward up_down←↓⍉↑⍵ ⍝ forward is firsts, up_down is seconds 
  aim←+\up_down        ⍝ aim is sum of ups and downs
  depth←+/aim×forward  ⍝ depths are sum of aim times forward
  pos←+/forward        ⍝ pos is sum of forward
  depth×pos
}

In [120]:
day2_2 test2

900

## Day 3

[Puzzle link](https://adventofcode.com/2021/day/3)

Today our data is a list of binary numbers we need to look at bitwise,
so we'll want that as a bit matrix.

In [155]:
⎕←test3←↑⍎¨¨⊃⎕NGET 'day3-test.txt' 1 ⍝ mix (unbox as rows) eval'd digits from lines

0 0 1 0 0
1 1 1 1 0
1 0 1 1 0
1 0 1 1 1
1 0 1 0 1
0 1 1 1 1
0 0 1 1 1
1 1 1 0 0
1 0 0 0 0
1 1 0 0 1
0 0 0 1 0
0 1 0 1 0

In [123]:
data3←↑⍎¨¨⊃⎕NGET 'day3-input.txt' 1

### Part 1

We're supposed to form two binary numbers by taking the most and least
commmon bits each in each position. We can sum bits in each column to
find the most common bit, e.g.

In [129]:
(2÷⍨≢test3)<+⌿test3 ⍝ bit mask where 6 < column sums of test3

1 0 1 1 0

APL has fancy operators for base conversion too so this can be smushed
into another ridiculous one liner. It could even probably be _more_ condensed,
in case it's already too readable.

In [151]:
day3_1←2⊥((2÷⍨(≢⊢))<+⌿)×2⊥(2÷⍨(≢⊢))>+⌿  ⍝ sorry

In [152]:
day3_1 test3

198

In [153]:
day3_1 data3

3429254

### Part 2

Now we gotta find a row in our bit matrix, by taking only rows with the majority
bit in the first position, then from those only rows with the majority bit in
the second position, and so on until we only have one row left. This requires
iteratively filtering a set of things which, unsurprisingly, we can do in one
line of terrible code. (Sorry.)

Finding the majority bit in the leftmost position is easy. Using ≤ means that
when there are an equal number of 0s and 1s we prefer 1.

In [156]:
⊃((2÷⍨(≢⊢))≤+⌿)test3  ⍝ take first from bit mask where 6 ≤ column sums

1

Here is one way to get the first (leftmost) bit of each row.

In [158]:
⊣/test3  ⍝ i.e. first column

0 1 1 1 1 0 0 1 1 1 0 0

In [163]:
(⊃((2÷⍨(≢⊢))≤+⌿)test3)=⊣/test3  ⍝ bit mask of rows matching majority bit...

0 1 1 1 1 0 0 1 1 1 0 0

In [164]:
((⊃((2÷⍨(≢⊢))≤+⌿)test3)=⊣/test3)⌿test3  ⍝ select just the matching rows

1 1 1 1 0
1 0 1 1 0
1 0 1 1 1
1 0 1 0 1
1 1 1 0 0
1 0 0 0 0
1 1 0 0 1

Now we just need to rotate the masks and then iterate this n times
where n is the number of bits.

In [165]:
1⌽((⊃((2÷⍨(≢⊢))≤+⌿)test3)=⊣/test3)⌿test3  ⍝ rotate left

1 1 1 0 1
0 1 1 0 1
0 1 1 1 1
0 1 0 1 1
1 1 0 0 1
0 0 0 0 1
1 0 0 1 1

In [166]:
({1⌽((⊃((2÷⍨(≢⊢))≤+⌿)⍵)=⊣/⍵)⌿⍵}⍣5)test3  ⍝ star dot means do 5 times

1 0 1 1 1

This idea almost works, except we have to deal with the case where there is
just one match left. Then we need to keep it for the next iteration whether or
not its current bit is 1 (the tiebreaker). Also, our result will actually be a
2d matrix with one row; the , operator converts it back down to a vector.

Now we can do this again for the minority bits and then convert to decimal
and multiply to get the answer.

In [227]:
o2_rating←,{({1⌽(((⊃((2÷⍨(≢⊢))≤+⌿)⍵)=⊣/⍵)∨((≢⍵)=1))⌿⍵}⍣(2⊃⍴⍵))⍵}

In [228]:
co2_rating←,{({1⌽(((⊃((2÷⍨(≢⊢))>+⌿)⍵)=⊣/⍵)∨((≢⍵)=1))⌿⍵}⍣(2⊃⍴⍵))⍵}

In [229]:
day3_2←2⊥o2_rating×2⊥co2_rating

In [231]:
day3_2 test3

230

In [232]:
day3_2 data3

5410338

## Day 4

[Puzzle link](https://adventofcode.com/2021/day/4)

We're playing bingo today. Calls are on the first input line, and
cards follow separated by blank lines. Rather than try to parse this
into one array and unbox it again later, we'll just split it out.

In [234]:
⎕←test4_calls←⍎¨','(≠⊆⊢)⊃⊃⎕NGET 'day4-test.txt' 1

7 4 9 5 11 17 23 2 0 14 21 24 10 16 13 6 15 25 12 22 18 20 8 19 3 26 1

In [235]:
⎕←test4_cards←↑↑⍎¨¨{((''≢⊢)¨⍵)⊆⍵}2↓⊃⎕NGET'day4-test.txt' 1

22 13 17 11 0
 8 2 23 4 24
21 9 14 16 7
 6 10 3 18 5
 1 12 20 15 19
 
 3 15 0 2 22
 9 18 13 17 5
19 8 7 25 23
20 11 10 24 4
14 21 16 12 6
 
14 21 17 24 4
10 16 15 9 19
18 8 23 26 20
22 11 13 6 5
 2 0 12 3 7

In [236]:
data4_calls←⍎¨','(≠⊆⊢)⊃⊃⎕NGET 'day4-input.txt' 1

In [237]:
data4_cards←↑↑⍎¨¨{((''≢⊢)¨⍵)⊆⍵}2↓⊃⎕NGET'day4-input.txt' 1

We'll be using 5x5 bit matrices to match called positions.
In this variant of bingo, diagonals don't count, so we just need
to sum up 1s in each row and column to detect bingos.

In [239]:
bingo←∨/5=((+/⊢),+⌿)  ⍝ or over boolean 5 = row sums cat col sums

### Part 1

We compute bit matrices of marked numbers for each prefix of calls.

In [349]:
5↑,\test4_calls  ⍝ e.g. scan cat to get prefixes of calls (5 shown for example)

┌─┬───┬─────┬───────┬──────────┐
│7│7 4│7 4 9│7 4 9 5│7 4 9 5 11│
└─┴───┴─────┴───────┴──────────┘

In [352]:
+⌿7 4 9∘.⍷test4_cards  ⍝ e.g. sum ⍷ to mark positions of 7 4 9

0 0 0 0 0
0 0 0 1 0
0 1 0 0 1
0 0 0 0 0
0 0 0 0 0
 
0 0 0 0 0
1 0 0 0 0
0 0 1 0 0
0 0 0 0 1
0 0 0 0 0
 
0 0 0 0 1
0 0 0 1 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

In [355]:
test4_game←{(+⌿(¯1,⍵)∘.⍷test4_cards)}¨,\test4_calls  ⍝ mark cards for each call prefix

Then we can find the first bingo round and board.

In [357]:
⊃1(⍸⍷)↑((bingo⍤2)¨test4_game)  ⍝ bingo over 2nd axis submatrices, find first one

12 3

Probably this could all go in some kind of horrible one liner but instead
I will put it into a dfn with names and things.

In [360]:
]dinput
day4_1←{
  cards calls←⍵
  game←{(+⌿(¯1,⍵)∘.⍷cards)}¨,\calls   ⍝ bit matrices for each prefix of calls 
  round card←⊃1(⍸⍷)↑((bingo⍤2)¨game)  ⍝ round and card of first bingo
  ⍝ score is winning call times sum of uncalled numbers on winning card
  (round⌷calls)×(+/+/(~card⌷↑round⌷game)×(card⌷cards))
}

In [361]:
day4_1 test4_cards test4_calls

4512

In [362]:
day4_1 data4_cards data4_calls

63424

### Part 2

Now we need to cheat instead and find the last winning card.

In [365]:
]dinput
day4_2←{
  cards calls←⍵
  game←{(+⌿(¯1,⍵)∘.⍷cards)}¨,\calls
  ⍝ last card to win and round in which it wins
  ⍝ (sum 1s for each non-winning row of col per card)
  round card←(⌈/,⊢⍳⌈/)1++⌿~↑((bingo⍤2)¨game)
  (round⌷calls)×(+/+/(~card⌷↑round⌷game)×(card⌷cards))
}

In [366]:
day4_2 test4_cards test4_calls

1924

In [367]:
day4_2 data4_cards data4_calls

23541

## Day 5

[Puzzle link](https://adventofcode.com/2021/day/5)

Today we get to plot lines. Splitting on ',' and ' -> ' is a little
different since usually we split on one character; ∊ gives an
array with 1s for delimiter chars ~ flips it for partition.

In [369]:
⎕←test5←↑⍎¨¨{', -> '(~⍤∊⍨⊆⊢)⍵}¨⊃⎕NGET'day5-test.txt' 1

0 9 5 9
8 0 0 8
9 4 3 4
2 2 2 1
7 0 7 4
6 4 2 0
0 9 2 9
3 4 1 4
0 0 8 8
5 5 8 2

In [370]:
data5←↑⍎¨¨{', -> '(~⍤∊⍨⊆⊢)⍵}¨⊃⎕NGET'day5-input.txt' 1

### Part 1

We are supposed to consider just horizontal or vertical lines.

In [371]:
h_or_v←{((1⌷⍵)=(3⌷⍵))∨((2⌷⍵)=(4⌷⍵))}  ⍝ same x or y

To find all the intersections we'll just plot them all.

In [376]:
]dinput
day5_1←{
  ⍝ counts overlapping points for horizontal and vertical lines only 
  grid←1000 1000⍴0
  plot_hv←{
    ⍝ plots a horizontal or vertical line in grid
    line←⍵                  ⍝ x1 y1 x2 y2
    p1←line[1 2]⌊line[3 4]  ⍝ p1 is lower leftmost
    p2←line[1 2]⌈line[3 4]  ⍝ p2 is upper rightmost
    pts←(,⍳(1+p2-p1))+¨⊂p1  ⍝ indices from p1 to p2 inclusive
    grid[pts]+←1            ⍝ plot the points
    ⍬
  }
  hv←((h_or_v⍤1)⍵)⌿⍵  ⍝ reduce lines over h_or_v on each row
  _←plot_hv¨↓hv       ⍝ plot the lines
  (+/,)grid≥2         ⍝ sum where grid ≥ 2
}

In [377]:
day5_1 test5

5

In [378]:
day5_1 data5

4655

### Part 2

Now we are supposed to consider all the lines. This is actually a
little easier. The trickiest bit is just generating the plot indices.
The key idea is that ×p2-p1 is a unit vector from p1 to p2 (scalar ×
is the sign function).

In [379]:
]dinput
day5_2←{
  ⍝ counts overlapping points for horizontal, vertical, and diagonal lines
  grid←1000 1000⍴0
  plot←{
    ⍝ plots a line in the grid
    line←⍵
    p1←line[1 2]
    p2←line[3 4]
    ⍝ iota length times unit vector
    ds←((⍳(1+⌈/|p2-p1))-1)∘.×(×p2-p1)
    pts←ds+((⍴ds)⍴(1 1+p1))  ⍝ add back p1
    grid[↓pts]+←1
    ⍬
  }
  _←plot¨↓⍵   ⍝ plot the lines
  (+/,)grid≥2 ⍝ sum where grid ≥ 2
}

In [380]:
day5_2 test5

12

In [381]:
day5_2 data5

20500

## Day 6

[Puzzle link](https://adventofcode.com/2021/day/6)

Today we have to simulate fish reproduction. Great.

In [383]:
test6←3 4 3 1 2

In [396]:
data6←⊃⍎¨¨','(≠⊆⊢)¨⊃⎕NGET'day6-input.txt' 1

### Part 1

For small numbers of fish this is just a one liner. We concatenate
new 8s for each 0, replace 0s with 7s, and decrement everything so
really 0s become 6s. The starfish operator iterates (appropriately).

In [390]:
day6_1←≢({(7×(0=⍵))+(⍵-1)},{8+⍵/⍨(0∘=)⍵})⍣80

In [391]:
day6_1 test6

5934

In [397]:
day6_1 data6

377263

### Part 2

All right, 256 days is too many to cheese this. We will keep a census
of fish in the population instead of storing one number per fish.

In [399]:
]dinput
day6_2←{
  ⍝ (n day6_2 fish) returns population size after n generations
  n←⍺
  census←9⍴0     ⍝ init to zero of each age
  census[1+⍵]+←1 ⍝ add 1 to count of age for each fish
  ⍝ 1⊖ rotates the current ages down 1
  generation←{(1⊖⍵)+(0 0 0 0 0 0,⍵[1],0 0)}
  +/(generation⍣n)census
}

In [403]:
⎕PP←16  ⍝ we need more precision, apparently

In [404]:
256 day6_2 test6

26984457539

In [405]:
256 day6_2 data6

1695929023803

## Day 7

[Puzzle link](https://adventofcode.com/2021/day/7)

Something with crabs. I can't even with this today.

In [431]:
⎕←test7←⍎¨','(≠⊆⊢)⊃⊃⎕NGET 'day7-test.txt' 1

16 1 2 0 4 2 7 1 2 14

In [432]:
data7←⍎¨','(≠⊆⊢)⊃⊃⎕NGET 'day7-input.txt' 1

### Part 1

Find the minimum fuel cost for all crabs to align. It makes no sense
to move away from all the other crabs so this will clearly occur at a
point inside the convex hull of the crabs.

For part 1 this should occur at some crab's current position. Consider
a non-crab position with M crabs to the left and N crabs to the right,
where the nearest crab in either direction is m or n units away.
Choosing the nearest left crab's position instead would reduce cost by
δ=(M-N)m; right would reduce by δ=(N-M)n. m>0, n>0, and either M-N≥0 or
N-M≥0 or both so this is the same or better.

In [433]:
day7_1←{v←⍵ ⋄ ⌊/{+/(|⍵-⍨v)}¨v}

In [434]:
day7_1 test7

37

In [435]:
day7_1 data7

357353

### Part 2

The new cost function uses triangular numbers.

In [462]:
tri←{(⍵×(⍵+1))÷2}

The minimum fuel cost isn't at some crab's current position anymore.
For example, now it costs 1+1 for x=0 and x=2 to meet at x=1, but it
would cost 1+2 for them to meet at x=0 or x=2. But the minimum is
still in the convex hull, so just search over the whole range. 

In [463]:
day7_2←{v←⍵ ⋄ ⌊/{+/tri(|⍵-v)}¨⍳⌈/v}

In [464]:
day7_2 test7

168

In [465]:
day7_2 data7

104822130

## Day 8

[Puzzle link](https://adventofcode.com/2021/day/8)

We're sorting out really specifically mangled wiring today.

In [467]:
⎕←test8←↑'|'(≠⊆⊢)¨⊃⎕NGET'day8-test.txt' 1  ⍝ extra spaces are ok

┌───────────────────────────────────────────────────────────┬──────────────────
│be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb │ fdgacbe cefdb cef
├───────────────────────────────────────────────────────────┼──────────────────
│edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec │ fcgedb cgb dgebac
├───────────────────────────────────────────────────────────┼──────────────────
│fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef │ cg cg fdcagb cbg 
├───────────────────────────────────────────────────────────┼──────────────────
│fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega │ efabcd cedba gadf
├───────────────────────────────────────────────────────────┼──────────────────
│aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga │ gecf egdcabf bgf 
├───────────────────────────────────────────────────────────┼──────────────────
│fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf │ gebdcfa ecba ca f
├───────────────────────────────────────────────────────────┼──────────────────
│dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf │ cefg dcbef fcge g
├───────────────────────────────────────────────────────────┼──────────────────
│bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd │ ed bcgafe cdgba c
├───────────────────────────────────────────────────────────┼──────────────────
│egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg │ gbdfcae bgc cg cg
├───────────────────────────────────────────────────────────┼──────────────────
│gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc │ fgae cfgab fg bag
└───────────────────────────────────────────────────────────┴──────────────────

 ────────┐
 bgd gcbe│
 ────────┤
 f gc │
 ────────┤
 │
 ────────┤
 ec cb │
 ────────┤
 bfgea │
 ────────┤
 adegcb │
 ────────┤
 bcadfe │
 ────────┤
 bgef │
 ────────┤
 b │
 ────────┤
 ce │
 ────────┘

In [469]:
data8←↑'|'(≠⊆⊢)¨⊃⎕NGET'day8-input.txt' 1

In theory one could probably do this with boolean algebra and logic.  But
we are absolutely going to cheese this instead, so will be needing a way to
generate permutations. I got this from APLcart. Instead of thinking about
fancy lightbulbs, let's think about how it works.

In [470]:
permutations←{(⍳⍵)(,[⍳2](⊢,⍤1 0~)⍤1)⍣⍵⍉⍪⍬}

It is iterating something ⍵ times on ⍳⍵ and ⍉⍪⍬. That something is ,[⍳2](⊢,⍤1 0~).

In [471]:
,[⍳2](⊢,⍤1 0~)⍤1

┌──┴──┐ 
 [1·2] ⍤ 
┌─┘ ┌─┴─┐
, ┌─┼─┐ 1
 ⊢ ⍤ ~ 
 ┌┴┐ 
 , 1 0

,⍤1 0 pairs the left vector with each element from the right; (⊢(,⍤1 0)~) instead pairs the right vector with each element from the left less those from the right.  ,[⍳2] is catenate with an axis specification [1 2] presumably to put things into rows.

In [486]:
⎕←d8p1←1 2 3(,[⍳2](⊢,⍤1 0~)⍤1)⍉⍪⍬

1
2
3

In [488]:
⎕←d8p2←1 2 3(,[⍳2](⊢,⍤1 0~)⍤1)d8p1

1 2
1 3
2 1
2 3
3 1
3 2

In [489]:
1 2 3(,[⍳2](⊢,⍤1 0~)⍤1)d8p2

1 2 3
1 3 2
2 1 3
2 3 1
3 1 2
3 2 1

So this is iteratively catenating new rows with each element of ⍳3 not in the left. Kinda mind blowing.

### Part 1

For part 1 we are just supposed to count up strings of certain lengths...

In [490]:
day8_1←{+/(2 3 4 7)∊⍨≢¨,↑' '(≠⊆⊢)¨(1∘↓⍤1)⍵}

In [491]:
day8_1 test8

26

In [492]:
day8_1 data8

470

### Part 2

Now we have to figure out what's up with the wiring. We first define the
canonical set of segments for each digit from 0 to 9.

In [493]:
digits←'abcefg' 'cf' 'acdeg' 'acdfg' 'bcdf' 'abdfg' 'abdefg' 'acf' 'abcdefg' 'abcdfg'

We will also need a helper which translates corresponding values from one array to another, also from APLcart.

In [494]:
translate←{⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵}

In [495]:
('12345'translate'abcde')'52341'

ebcda

In [496]:
permutations_of_length_7←permutations 7

To decode the scrambled up displays, we will just try all 5040 permutations
on the scrambled digit set until we get the canonical digit set.

In [502]:
]dinput
decode_display←{
  patterns←{⍵[⍋⍵]}¨' '(≠⊆⊢)⊃(1↑⍵)  ⍝ scrambled segment sets for each digit
  readout←{⍵[⍋⍵]}¨' '(≠⊆⊢)⊃(1↓⍵)   ⍝ segment sets we are supposed to decode
  trial←{
    assignment←'abcdefg'[⊃⍵]  ⍝ segment permutation to assume
    ⍝ find index of each remapped pattern in canonical digits
    code←digits⍳({⍵[⍋⍵]}¨(assignment translate'abcdefg')¨patterns)
    ⍝ if all digits found, decode the readout 
    (∧/code≤10):10⊥code[patterns⍳readout]-1
    ((≢⍵)=0):'error'
    ∇(1↓⍵)
  }
  trial(↓permutations_of_length_7)
}

In [499]:
day8_2←{+/decode_display¨↓⍵}

In [500]:
day8_2 test8

61229

In [501]:
day8_2 data8

989396

## Day 9

[Puzzle link](https://adventofcode.com/2021/day/9)

Woo hoo, today looks really array-ish.

In [504]:
⎕←test9←↑⍎¨¨⊃⎕NGET 'day9-test.txt' 1

2 1 9 9 9 4 3 2 1 0
3 9 8 7 8 9 4 9 2 1
9 8 5 6 7 8 9 8 9 2
8 7 6 7 8 9 6 7 8 9
9 8 9 9 9 6 5 6 7 8

In [793]:
data9←↑⍎¨¨⊃⎕NGET 'day9-input.txt' 1

### Part 1

APL's ⌺ operator lets us count in neighborhoods really easily.
Technically this problem doesn't include diagonal neighbors but it
works anyways because of what we're doing and I reaaallly wanna use
it, ok?

We find a bit matrix of all the 3x3s where the center element is
≤ all its neighbors, dropping those that are padding. Then get the
values at those indices and add 1 and sum reduce. Whee...

In [507]:
day9_1←{+/1+⍵[⍸(∧/¨,¨({⊂⍺↓((2 2)⌷⍵)≤⍵}⌺3 3⊢⍵))]}

In [508]:
day9_1 test9

15

In [509]:
day9_1 data9

545

### Part 2

We have to count the sizes of connected regions in an array. Originally I just
solved this with a chesy flood fill, but I was hoping there would be a clever
morphological image processing way to do this.

The idea would be to build a bit matrix and have the 1s "flow" to
adjacent cells. But there's ambiguity about which way they should flow. For
example, the test has an 8 surrounded by two 7s. The flow could "split",
but then that's nonlocal. So I guess I'll stick with the flood fill.

In [789]:
]dinput
flood_fill←{
  to_visit←⊃1⌷⍵
  visited←⊃2⌷⍵
  regions←⊃3⌷⍵
  ((≢to_visit)=0):visited
  next←1↑to_visit
  already_visited←∨/∨/next⍷visited
  outside←next{3::1 ⋄ 0=((⊃⍺)⌷↑⍵)}regions
  (already_visited∨outside):∇((1↓to_visit)visited regions)
  nextp←(next+((1 0)(0 1)(¯1 0)(0 ¯1))),1↓to_visit
  visitedp←visited,next
  ∇(nextp visitedp regions)
}

In [790]:
]dinput
day9_2←{
  sinks←{⍸(∧/¨,¨({⊂⍺↓((2 2)⌷⍵)≤⍵}⌺3 3⊢⍵))}⍵  ⍝ sink positions
  basins←~9=⍵  ⍝ bitmask of basins
  region_size←≢1↓{flood_fill(⊂⍵)(⊂⍬)(⊂basins)}
  sizes←region_size¨sinks
  ×/sizes[3↑⍒sizes]
}

In [791]:
day9_2 test9

1134

In [794]:
day9_2 data9

950600

## Day 10

[Puzzle link](https://adventofcode.com/2021/day/10)

Parsing stuff today. Not really APL's strong suit.

In [796]:
⎕←test10←⊃⎕NGET 'day10-test.txt' 1

┌────────────────────────┬──────────────────────┬────────────────────────┬─────
│[({(<(())[]>[[{[]{<()<>>│[(()[<>])]({[<{<<[]>>(│{([(<{}[<>[]}>{[]{[(<()>│(((({
└────────────────────────┴──────────────────────┴────────────────────────┴─────

 ──────────────────┬──────────────────────┬───────────────────────┬───────
 <>}<{<{<>}{[]{[]{}│[[<[([]))<([[{}[[()]]]│[{[{({}]{}}([{[{{{}}([]│{<[[]]>
 ──────────────────┴──────────────────────┴───────────────────────┴───────

 ────────────────┬──────────────────────┬──────────────────────┬──────────
 }<{[{[{[]{()[[[]│[<(<(<(<{}))><([]([]()│<{([([[(<>()){}]>(<<{{│<{([{{}}[<
 ────────────────┴──────────────────────┴──────────────────────┴──────────

 ──────────────┐
 [[[<>{}]]]>[]]│
 ──────────────┘

In [797]:
data10←⊃⎕NGET 'day10-input.txt' 1

There's nothing particularly hard here but Dyalog really liked to crash with
a mysterious error 139 a lot. So some of the code is a little awkward because
I kept modifying it not to break the interpreter.

In [799]:
]dinput
parse_chunks←{
  expr←⍵
  stack←⍬
  next_char←{
    i←⍵
    i>≢expr:0
    c←expr[i]
    ⍝ for opening chars, push the expected closing char
    _←{c='[':stack,←']' ⋄ 0}0
    _←{c='{':stack,←'}' ⋄ 0}0
    _←{c='(':stack,←')' ⋄ 0}0
    _←{c='<':stack,←'>' ⋄ 0}0
    c∊'[{(<':∇(i+1)
    ⍝ for closing chars, check if they match
    match←{
      top←¯1↑stack
      stack⊢←¯1↓stack
      c=top
    }0
    match:∇(i+1)  ⍝ char matches keep going
    ⍝ wrong closing char score it
    c=')':3
    c=']':57
    c='}':1197
    c='>':25137
  }
  next_char 1
}

### Part 1

Score the mismatched closing chars.

In [800]:
day10_1←+/(parse_chunks¨⊢)

In [801]:
day10_1 test10

26397

In [802]:
day10_1 data10

343863

### Part 2

Now we need to add up the stack contents instead.

In [804]:
]dinput
parse_chunks2←{
  expr←⍵
  stack←⍬
  next_char←{
    i←⍵
    i>≢expr:0
    c←expr[i]
    ⍝ for opening chars, push the expected closing char
    _←{c='[':stack,←']' ⋄ 0}0
    _←{c='{':stack,←'}' ⋄ 0}0
    _←{c='(':stack,←')' ⋄ 0}0
    _←{c='<':stack,←'>' ⋄ 0}0
    c∊'[{(<':∇(i+1)
    ⍝ for closing chars, check if they match
    match←{
      top←¯1↑stack
      stack⊢←¯1↓stack
      c=top
    }0
    match:∇(i+1)  ⍝ char matches keep going
    1  ⍝ this is a corrupted line skip it
  }
  corrupt←next_char 1
  corrupt:0
  ⍝ score the expected completion if not corrupt
  5⊥{⍵=')':1 ⋄ ⍵=']':2 ⋄ ⍵='}':3 ⋄ ⍵='>':4}¨⌽stack
}

In [815]:
]dinput
day10_2←{
  scores←(0∘≠){⍵/⍨⍺⍺ ⍵}parse_chunks2¨⍵  ⍝ nonzero scores
  ({(⊂⍋⍵)⌷⍵}scores)[⌈(≢scores)÷2]       ⍝ middle of sorted scores
}

In [816]:
day10_2 test10

288957

In [817]:
day10_2 data10

2924734236

## Day 11

[Puzzle link](https://adventofcode.com/2021/day/11)

Cool trippy looking octopus simulation today.

In [819]:
⎕←test11←↑↑⍎¨¨⊃⎕NGET 'day11-test.txt' 1

5 4 8 3 1 4 3 2 2 3
2 7 4 5 8 5 4 7 1 1
5 2 6 4 5 5 6 1 7 3
6 1 4 1 3 3 6 1 4 6
6 3 5 7 3 8 5 4 7 8
4 1 6 7 5 2 4 6 4 5
2 1 7 6 8 4 1 7 2 1
6 8 8 2 8 8 1 1 3 4
4 8 4 6 8 4 8 5 5 4
5 2 8 3 7 5 1 5 2 6

In [820]:
data11←↑↑⍎¨¨⊃⎕NGET 'day11-input.txt' 1

We need a little dfn to simulate the octopi.

In [821]:
]dinput
step_octopi←{
  grid←1+⍵
  already_flashed←(⍴grid)⍴0
  do_flashes←{
    flash←(grid>9)∧~already_flashed
    ((+/+/flash)=0):grid
    grid+←{+/1↓¯5⌽,⍵}⌺3 3⊢flash
    already_flashed∨←flash
    ∇ 0
  }
  grid←do_flashes 0
  (~(grid>9))×grid
}

### Part 1

Count flashes after 100 steps.

In [824]:
]dinput
day11_1←{
  flashes←0
  n←⍺
  grid←⍵
  (n=0):flashes
  grid←step_octopi grid
  flashes+←+/+/0=grid
  flashes+(n-1)∇ grid
}

In [825]:
100 day11_1 test11

1656

In [826]:
100 day11_1 data11

1591

### Part 2

Find the step when all octopi flash.

In [829]:
]dinput
day11_2←{
  ⍺←1
  n←⍺
  grid←⍵
  grid←step_octopi grid
  ((×/⍴grid)=+/+/0=grid):n
  (n+1)∇ grid
}

In [830]:
day11_2 test11

195

In [831]:
day11_2 data11

314

## Day 12

[Puzzle link](https://adventofcode.com/2021/day/12)

Bleh graph search today. APL really isn't especially great at this.

I'll write out test12_1 longhand so that you can notice that 'x' is
a scalar and not a vector. But 'xx' is an array. So strings may be
arrays or not arrays depending on their length. This is somewhat
error-prone when trying things in the REPL.

In [839]:
⎕←test12_1←('start' (,'A'))('start' (,'b'))((,'A') (,'c'))((,'A') (,'b'))((,'b') (,'d'))((,'A') 'end')((,'b') 'end')

┌─────────┬─────────┬─────┬─────┬─────┬───────┬───────┐
│┌─────┬─┐│┌─────┬─┐│┌─┬─┐│┌─┬─┐│┌─┬─┐│┌─┬───┐│┌─┬───┐│
││start│A│││start│b│││A│c│││A│b│││b│d│││A│end│││b│end││
│└─────┴─┘│└─────┴─┘│└─┴─┘│└─┴─┘│└─┴─┘│└─┴───┘│└─┴───┘│
└─────────┴─────────┴─────┴─────┴─────┴───────┴───────┘

In [833]:
⎕←test12_2←'-'(≠⊆⊢)¨⊃⎕NGET'day12-test2.txt' 1

┌────────┬──────────┬──────────┬──────────┬───────┬───────┬────────┬───────┬───
│┌──┬───┐│┌──┬─────┐│┌─────┬──┐│┌──┬─────┐│┌──┬──┐│┌──┬──┐│┌──┬───┐│┌──┬──┐│┌──
││dc│end│││HN│start│││start│kj│││dc│start│││dc│HN│││LN│dc│││HN│end│││kj│sa│││kj
│└──┴───┘│└──┴─────┘│└─────┴──┘│└──┴─────┘│└──┴──┘│└──┴──┘│└──┴───┘│└──┴──┘│└──
└────────┴──────────┴──────────┴──────────┴───────┴───────┴────────┴───────┴───

 ────┬───────┐
 ┬──┐│┌──┬──┐│
 │HN│││kj│dc││
 ┴──┘│└──┴──┘│
 ────┴───────┘

In [834]:
⎕←test12_3←'-'(≠⊆⊢)¨⊃⎕NGET'day12-test3.txt' 1

┌────────┬───────┬───────┬──────────┬───────┬────────┬───────┬───────┬───────┬─
│┌──┬───┐│┌──┬──┐│┌──┬──┐│┌─────┬──┐│┌──┬──┐│┌───┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌
││fs│end│││he│DX│││fs│he│││start│DX│││pj│DX│││end│zg│││zg│sl│││zg│pj│││pj│he│││
│└──┴───┘│└──┴──┘│└──┴──┘│└─────┴──┘│└──┴──┘│└───┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└
└────────┴───────┴───────┴──────────┴───────┴────────┴───────┴───────┴───────┴─

 ──────┬───────┬───────┬───────┬──────────┬───────┬───────┬───────┬───────
 ──┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌─────┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌─────┬
 RW│he│││fs│DX│││pj│RW│││zg│RW│││start│pj│││he│WI│││zg│he│││pj│fs│││start│
 ──┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└─────┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└─────┴
 ──────┴───────┴───────┴───────┴──────────┴───────┴───────┴───────┴───────

 ───┐
 ──┐│
 RW││
 ──┘│
 ───┘

In [835]:
data12←'-'(≠⊆⊢)¨⊃⎕NGET'day12-input.txt' 1

### Part 1

Count all paths visiting small letter caves at most once. We'll just use
a recursive dfn to enumerate them.

In [845]:
]dinput
paths←{
  graph←⍵
  xs←⊃¨graph
  ys←2⊃¨graph
  is_lower←∧/(1∘⎕C≠⊢)  ⍝ 1 if c is lowercase
  visit←{
    cur←⊃⍵[1]  ⍝ current node
    pps←⊃⍵[2]  ⍝ open partial paths
    (cur≡'end'):(⊂'end')∘,¨pps  ⍝ open paths stop at end
    ⍝ collect adjacent vertices
    nexts←{(xs/⍨(,⍵)∘≡¨ys),(ys/⍨(,⍵)∘≡¨xs)}cur
    ⍝ prune paths that have gone through cur before if small 
    small←is_lower cur
    open_pps←pps/⍨((~small)∨(∧/¨(,cur∘≢¨¨pps)))
    ((≢open_pps)=0):0  ⍝ dead end
    ⍝ return all the paths here
    next_pps←(⊂cur)∘,¨open_pps
    here←,↑{visit(⍵ next_pps)}¨nexts
    ((1∘≠)≢¨here)/here
  }
  ⍝ this filters out annoying empty paths
  paths_with_ghosts←visit('start'(,'-'))
  ('-'∊¨paths_with_ghosts)/paths_with_ghosts
}

In [846]:
day12_1←(≢paths)

In [847]:
day12_1 test12_1

10

In [848]:
day12_1 test12_2

19

In [849]:
day12_1 test12_3

226

In [850]:
day12_1 data12

3761

### Part 2

How many paths are there if we can revisit one small letter cave twice?

Originally, I modified my recursive dfn from part 1 to solve this part,
and it blew Dyalog's stack limit. So I rewrote the search to be iterative
instead. That worked but took so long that I wrote a naive Python solution
instead of waiting for it to finish (the runtime was 5 seconds).

I eventually figured out that "drop" is slow linear time and so it is really
easy to make things n^2 by mistake.

This uses "tradfns" and Dyalog's procedural programming keywords, which
feels super gross.

In [862]:
]dinput
 result←paths2_non_recursive graph;xs;ys;is_lower;q;n;i;cur;pps;open_pps;nexts;next_pps
 xs←⊃¨graph
 ys←2⊃¨graph
 is_lower←∧/(1∘⎕C≠⊢)

 paths_with_ghosts←⍬
 ⍝ trying to pop a queue is absurdly slow, so instead
 ⍝ keep our own running start index and length
 ⍝q←⊂('start'(,'-'))
 ⍝:While (≢q)>0
     ⍝cur←⊃⊃q
     ⍝pps←2⊃⊃q
     ⍝q←(1↓q)
 q←(,↓('start'(,'-')))
 n←1
 i←1
 :While n>0
     cur←⊃⊃q[i]
     pps←2⊃⊃q[i]
     n-←1
     i+←1
     :If (cur≡'end')
         paths_with_ghosts,←(⊂'end')∘,¨pps
         :Continue
     :EndIf
     open_pps←({cur is_open ⍵}¨pps)/pps
     :If (≢open_pps)=0
         :Continue
     :EndIf
     nexts←{(xs/⍨(,⍵)∘≡¨ys),(ys/⍨(,⍵)∘≡¨xs)}cur
     next_pps←(⊂cur)∘,¨open_pps
     q,←{(⍵ next_pps)}¨nexts
     n+←≢nexts
 :EndWhile
 result←('-'∊¨paths_with_ghosts)/paths_with_ghosts

I extracted a separate procedure for is_open in an effort to see
if it would speed up the search. It did not.

In [863]:
 ]dinput
 result←cur is_open pp;ppt;count
 :If (~(is_lower cur))
     result←1
     :Return
 :EndIf
 :If ((cur≡'start')∧((≢pp)>1))
     result←0
     :Return
 :EndIf
 ppt←(⊂cur),pp
 count←{(≢⍵)}⌸((is_lower¨ppt)/ppt)
 :If (∨/(>∘2)count)
     result←0
     :Return
 :EndIf
 :If ((+/(=∘2)count)>1)
     result←0
     :Return
 :EndIf
 result←1

In [856]:
day12_2←(≢paths2_non_recursive)

In [864]:
day12_2 test12_1

36

In [865]:
day12_2 test12_2

103

In [866]:
day12_2 test12_3

3509

In [860]:
⍝day12_2 data12  ⍝ still really slow

## Day 13

[Puzzle link](https://adventofcode.com/2021/day/13)

Ooh, origami today, that sounds cool.

In [53]:
test13 ← ⊃⎕NGET 'day13-test.txt' 1

In [70]:
data13 ← ⊃⎕NGET 'day13-input.txt' 1

Let's start by parsing out the points and the folding instructions.

In [421]:
]dinput
parse_folds←{
  ⍝ given lines like ('0,0')('3,2')('fold along x=5')('fold along y=4')
  ⍝ returns (points folds) where points are (x y)s and folds are (x 0)
  ⍝ for vertical folds and (0 y) for horizontal folds 
  parse_line←{
    ⍝ note that we add 1 to x ys because APL arrays are 1-indexed
    ','∊⍵:1,(⌽1+⍎¨','(≠⊆⊢)⍵) ⍝ ',' lines: 1 x+1 y+1 
    'x'∊⍵:2(⍎2⊃'='(≠⊆⊢)⍵)0   ⍝ 'x' lines: 2 x 0
    'y'∊⍵:2 0(⍎2⊃'='(≠⊆⊢)⍵)  ⍝ 'y' lines: 2 0 y
    0                        ⍝ ignore extra newlines
  }
  ⍝ selects rows with first element =⍺, dropping that element
  select←{1↓¨(⍺=⊃¨⍵)/⍵}
  ⍝ collect 1 lines (i.e. points) and then 2 lines (i.e. folds)
  (⊂∘(1∘select),⊂∘(2∘select))parse_line¨⍵
}

We can sanity check this with a little plotting helper.

In [79]:
]dinput
plot←{
  points←⍵
  paper←(↑⌈/points)⍴0  ⍝ 0s as wide and tall as max x and max y
  paper[points]←1      ⍝ plot 1s at each point
  paper
}

In [422]:
⎕←x←plot ⊃parse_folds test13

0 0 0 1 0 0 1 0 0 1 0
0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 1 0 1
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 1 0 1 1 0
0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 1
1 0 0 0 0 0 0 0 0 0 0
1 0 1 0 0 0 0 0 0 0 0

To fold, we can just reflect the matrix horizontally or vertically
and or it with itself, and then take only the upper or left part.

In [81]:
⎕←x←7↑x∨⊖x   ⍝ e.g. fold up and keep 7 rows

1 0 1 1 0 0 1 0 0 1 0
1 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 1
1 0 0 0 1 0 0 0 0 0 0
0 1 0 1 0 0 1 0 1 1 1
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0

In [82]:
⎕←x←5↑[2]x∨⌽x ⍝ now fold left and keep 5 cols

1 1 1 1 1
1 0 0 0 1
1 0 0 0 1
1 0 0 0 1
1 1 1 1 1
0 0 0 0 0
0 0 0 0 0

Let's just write a general program for any number of folds and then
have it only do the first to solve part 1. It isn't really that much
different.

In [83]:
]dinput
fold←{
  points folds←⍵
  paper←plot points ⍝ make bitmap of points
  fold_left←{
    ⍝ modify paper, or'ing right-to-left and keeping left ⍵ cols
    paper⊢←⍵↑[2]paper∨⌽paper ⋄ 0
  }
  fold_up←{
    ⍝ modify paper, or'ing top-to-bottom and keeping top ⍵ rows
    paper⊢←⍵↑paper∨⊖paper ⋄ 0
  }
  _←{
    left up←⍵   ⍝ each element has x/y fold amount  
    (left≠0):fold_left left
    fold_up up
  }¨folds
  paper
}

In [84]:
fold parse_folds test13

1 1 1 1 1
1 0 0 0 1
1 0 0 0 1
1 0 0 0 1
1 1 1 1 1
0 0 0 0 0
0 0 0 0 0

### Part 1

To prove this works, count up points after doing just the first fold.
The inline dfn here keeps all the points and only the first fold from
instructions.

In [72]:
day13_1←{+/+/fold{(⊃⍵)(⊂(⊃2⊃⍵))}parse_folds ⍵}

In [85]:
day13_1 test13

17

In [86]:
day13_1 data13

842

### Part 2

Then we do the whole thing and get a bitmap to answer with.

In [88]:
day13_2 ← fold parse_folds

In [89]:
day13_2 data13

1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 0
1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0
1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0
1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0
1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0
1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 0

## Day 14

Today we get to do string shenanigans, substituting pairs for letters.
APL doesn't really seem to have dictionaries so I guess I'll roll my own...

In [242]:
⎕←polymers←↑('NN' (,'C'))('NC' (,'B'))('CB' (,'H'))

┌──┬─┐
│NN│C│
├──┼─┤
│NC│B│
├──┼─┤
│CB│H│
└──┴─┘

In [245]:
↑polymers[⍸(⌽'NC'∘≡¨polymers)]  ⍝ lookup letter where adjacent column is 'NC'

B

In [246]:
2,/'NNCB'  ⍝ get pairs from string

┌──┬──┬──┐
│NN│NC│CB│
└──┴──┴──┘

In [249]:
{⊃⊃polymers[⍸(⌽⍵∘≡¨polymers)]}¨2,/'NNCB'  ⍝ lookup letter for each pair

CBH

We're supposed to squidge the letters between the pairs.

In [250]:
{(⊃,/{(⊃⍵),⊃⊃polymers[⍸(⌽⍵∘≡¨polymers)]}¨2,/⍵),¯1↑⍵}'NNCB'  ⍝ put pair letters around lookups

NCNBCHB

In [263]:
]dinput
polymerize←{
  pairs←⍺
  template←⍵
  lookup←{⊃⊃pairs[⍸(⌽⍵∘≡¨pairs)]}
  (⊃,/{(⊃⍵),lookup⍵}¨2,/template),¯1↑template
}

In [264]:
polymers polymerize 'NNCB'

NCNBCHB

Ok, time to read inputs.

In [258]:
⎕←test14_template←⊃⊃⎕NGET'day14-test.txt' 1

NNCB

In [259]:
⎕←test14_pairs←↑{' -> '(~⍤∊⍨⊆⊢)⍵}¨2↓⊃⎕NGET'day14-test.txt' 1

┌──┬─┐
│CH│B│
├──┼─┤
│HH│N│
├──┼─┤
│CB│H│
├──┼─┤
│NH│C│
├──┼─┤
│HB│C│
├──┼─┤
│HC│B│
├──┼─┤
│HN│C│
├──┼─┤
│NN│C│
├──┼─┤
│BH│H│
├──┼─┤
│NC│B│
├──┼─┤
│NB│B│
├──┼─┤
│BN│B│
├──┼─┤
│BB│N│
├──┼─┤
│BC│B│
├──┼─┤
│CC│N│
├──┼─┤
│CN│C│
└──┴─┘

In [287]:
data14_template←⊃⊃⎕NGET'day14-input.txt' 1

In [288]:
data14_pairs←↑{' -> '(~⍤∊⍨⊆⊢)⍵}¨2↓⊃⎕NGET'day14-input.txt' 1

### Part 1

Polymerize repeatedly...

In [274]:
((test14_pairs∘polymerize)⍣4) test14_template

NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB

and find the most common minus the least common character.

In [280]:
most_minus_least←(⌈/-⌊/){≢⍵}⌸∘⊢  ⍝ tally keys, then take max-min 

In [283]:
most_minus_least 'ABBBCCCCC'

4

In [284]:
day14_1←{most_minus_least ((⍺∘polymerize)⍣10) ⍵}

In [285]:
test14_pairs day14_1 test14_template

1588

In [289]:
data14_pairs day14_1 data14_template

2587

### Part 2

Now we have to do this 40 times in a row. Naive string substitution
won't work anymore; Dyalog bogs down at around 16 or 17 iterations.

Notice that in each generation, each pair yields 2 new pairs: xy -> z
gives xz and zy. The order isn't important, so we can just count them.
First we have to count pairs in the template.

In [310]:
count_pairs←{⍵ 1}¨2,/⊢  ⍝ (pair 1) for each pair in template

In [311]:
test14_census←count_pairs test14_template

In [329]:
⎕←data14_census←count_pairs data14_template

┌──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬─
│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌
││VH│1│││HC│1│││CK│1│││KB│1│││BF│1│││FO│1│││OV│1│││VC│1│││CH│1│││HH│1│││HK│1│││
│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└
└──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴─

 ─────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┐
 ──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│
 KO│1│││OH│1│││HB│1│││BP│1│││PN│1│││NC│1│││CK│1│││KO│1││
 ──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│
 ─────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┘

Note that CK and KO are duplicated, so we need to collect counts.

In [331]:
merge_counts←{c←⍵ ⋄ ↓{(⊃⍺)(+/⊢/↑c[⍵])}⌸⊣/↑c}  ⍝ sum counts for duplicate keys

In [332]:
merge_counts data14_census

┌──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬─
│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌
││VH│1│││HC│1│││CK│2│││KB│1│││BF│1│││FO│1│││OV│1│││VC│1│││CH│1│││HH│1│││HK│1│││
│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└
└──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴─

 ─────┬──────┬──────┬──────┬──────┬──────┐
 ──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│
 KO│2│││OH│1│││HB│1│││BP│1│││PN│1│││NC│1││
 ──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│
 ─────┴──────┴──────┴──────┴──────┴──────┘

We can now polymerize pair counts rather than strings, which ought to be
much more efficient.

In [345]:
]dinput
polymerize2←{
  ⍝ update association list of (pair count) by inserting two new pairs
  ⍝ for each existing pair according to substitution instructions
  pairs←⍺   ⍝ (pair letter) substitution instructions
  census←⍵  ⍝ (pair count) current counts
  lookup←{⊃⊃pairs[⍸(⌽⍵∘≡¨pairs)]}
  merge_counts,↑{
    pair count←⍵
    letter←lookup pair
    left←(1↑pair),letter
    right←letter,(¯1↑pair)
    (left count)(right count)
  }¨census
}

In [346]:
test14_pairs polymerize2 test14_census

┌──────┬──────┬──────┬──────┬──────┬──────┐
│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│
││NC│1│││CN│1│││NB│1│││BC│1│││CH│1│││HB│1││
│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│
└──────┴──────┴──────┴──────┴──────┴──────┘

But in the final analysis, we need to count letters, not pairs. Each
pair contributes one distinct letter except the last pair which
contributes two. We can refine merge to leave out the last pair.

In [333]:
merge_counts←{c←¯1↓⍵ ⋄ (↓{(⊃⍺)(+/⊢/↑c[⍵])}⌸⊣/↑c),(¯1↑⍵)}  ⍝ merge all but last pair counts

In [334]:
⎕←data14_census←merge_counts data14_census

┌──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬─
│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌
││VH│1│││HC│1│││CK│2│││KB│1│││BF│1│││FO│1│││OV│1│││VC│1│││CH│1│││HH│1│││HK│1│││
│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└
└──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴─

 ─────┬──────┬──────┬──────┬──────┬──────┬──────┐
 ──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│┌──┬─┐│
 KO│1│││OH│1│││HB│1│││BP│1│││PN│1│││NC│1│││KO│1││
 ──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│└──┴─┘│
 ─────┴──────┴──────┴──────┴──────┴──────┴──────┘

This way the last pair in the census will always be the literal last pair.

In [335]:
((test14_pairs∘polymerize2)⍣10) test14_census

┌────────┬────────┬────────┬────────┬───────┬────────┬───────┬────────┬───────┬
│┌──┬───┐│┌──┬───┐│┌──┬───┐│┌──┬───┐│┌──┬──┐│┌──┬───┐│┌──┬──┐│┌──┬───┐│┌──┬──┐│
││NB│796│││BB│812│││BN│735│││BC│120│││CC│60│││CN│102│││NC│42│││CB│114│││BH│81││
│└──┴───┘│└──┴───┘│└──┴───┘│└──┴───┘│└──┴──┘│└──┴───┘│└──┴──┘│└──┴───┘│└──┴──┘│
└────────┴────────┴────────┴────────┴───────┴────────┴───────┴────────┴───────┴

 ───────┬───────┬───────┬───────┬───────┬───────┬──────┐
 ┌──┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌──┬──┐│┌──┬─┐│
 │HC│76│││HH│32│││HN│27│││NH│27│││CH│21│││HB│26│││CB│1││
 └──┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└──┴──┘│└──┴─┘│
 ───────┴───────┴───────┴───────┴───────┴───────┴──────┘

Finally we just have to write a horrible function to sum up letter counts given
pair counts. It merges the counts of the first letters of each pair, plus the
last letter of the last pair, then takes the max count minus the min count.

In [336]:
letter_diff_from_census←{(⌈/-⌊/)⊢⌿⍉↑¯1↓(merge_counts ({((⊃⊃(⍵[1]))(⍵[2]))}¨⍵),(⊂((2⊃⊃⊃(¯1↑⍵)) 1)),0)}

In [337]:
letter_diff_from_census ((test14_pairs∘polymerize2)⍣10) test14_census

1588

In [338]:
letter_diff_from_census ((data14_pairs∘polymerize2)⍣10) data14_census

2587

Since it seems to work we can try it out on the problem.

In [339]:
day14_2 ← {letter_diff_from_census ((⍺∘polymerize2)⍣40) ⍵}

In [340]:
test14_pairs day14_2 test14_census

2.18819E12

In [342]:
⎕PP←16  ⍝ we need more precision, apparently

In [343]:
test14_pairs day14_2 test14_census

2188189693529

In [344]:
data14_pairs day14_2 data14_census

3318837563123

## Day 15

[Puzzle link](https://adventofcode.com/2021/day/15)

Find a shortest path from top left to bottom right in a grid.

In [406]:
⎕←test15←↑⍎¨¨⊃⎕NGET 'day15-test.txt' 1

1 1 6 3 7 5 1 7 4 2
1 3 8 1 3 7 3 6 7 2
2 1 3 6 5 1 1 3 2 8
3 6 9 4 9 3 1 5 6 9
7 4 6 3 4 1 7 1 1 1
1 3 1 9 1 2 8 1 3 7
1 3 5 9 9 1 2 4 2 1
3 1 2 5 4 2 1 6 3 9
1 2 9 3 1 3 8 5 2 1
2 3 1 1 9 4 4 5 8 1

In [407]:
data15←↑⍎¨¨⊃⎕NGET 'day15-input.txt' 1

We'll use [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm).
Here is straightforward code for it, not really particularly APLy.

In [409]:
]dinput
dijkstra←{
  grid←⍵
  in_bounds←{(∧/⍵≤(⊃⍴grid))∧(∧/⍵>0)}
  start←1 1           ⍝ start at upper left
  visited←(⍴grid)⍴0   ⍝ init all nodes to unvisited
  M←10000000          ⍝ this distance means "infinity"
  distance←(⍴grid)⍴M  ⍝ init nodes at "infinity"
  distance[⊂start]←0  ⍝ start at distance 0
  step←{
    m←distance+M×visited
    min←⌊/,m     ⍝ select min distance or M if all visited
    min≥M:0      ⍝ stop if all visited
    v←⊃⍸(m=min)  ⍝ find first index of min
    visited[⊂v]←1 ⍝ mark visited
    ⍝ collect in bounds, unvisited orthogonal neighbors
    neighbors←↓v(+⍤1)↑(¯1 0)(1 0)(0 ¯1)(0 1)
    neighbors←({~in_bounds ⍵:0 ⋄ ~visited[⊂⍵]}¨neighbors)/neighbors
    0=≢neighbors:∇ ⍬  ⍝ skip if no unvisited neighbors
    ⍝ update min distance to neighbors
    _←{distance[⊂⍵]⌊←grid[⊂⍵]+distance[⊂v]}¨neighbors
    ∇ ⍬
  }
  _←step ⍬
  distance
}

### Part 1

Just call our Dijkstra helper to get the shortest path length.

In [410]:
day15_1←(⍴⊢)⌷∘dijkstra⊢

In [411]:
day15_1 test15

40

In [412]:
day15_1 data15

458

### Part 2

The graph is secretly bigger and we need to tile it five times in each
direction and adjust the values slightly. We can do this easily with an
outer product over an index matrix ⍳5 5 and the tile.

In [414]:
⍳5 5

┌───┬───┬───┬───┬───┐
│1 1│1 2│1 3│1 4│1 5│
├───┼───┼───┼───┼───┤
│2 1│2 2│2 3│2 4│2 5│
├───┼───┼───┼───┼───┤
│3 1│3 2│3 3│3 4│3 5│
├───┼───┼───┼───┼───┤
│4 1│4 2│4 3│4 4│4 5│
├───┼───┼───┼───┼───┤
│5 1│5 2│5 3│5 4│5 5│
└───┴───┴───┴───┴───┘

In [417]:
1 2{1+9|((+/⍺)-2)+(⍵-1)}test15  ⍝ offset tile values by index

2 2 7 4 8 6 2 8 5 3
2 4 9 2 4 8 4 7 8 3
3 2 4 7 6 2 2 4 3 9
4 7 1 5 1 4 2 6 7 1
8 5 7 4 5 2 8 2 2 2
2 4 2 1 2 3 9 2 4 8
2 4 6 1 1 2 3 5 3 2
4 2 3 6 5 3 2 7 4 1
2 3 1 4 2 4 9 6 3 2
3 4 2 2 1 5 5 6 9 2

In [418]:
tile←{⊃⍪⌿,/(⍳5 5)∘.{1+9|((+/⍺)-2)+(⍵-1)}⊂⍵}  ⍝ flattened product of indices×adjusted tiles

In [419]:
day15_2←{t←tile ⍵ ⋄ (⍴t)⌷dijkstra t}

In [420]:
day15_2 test15

315

In [ ]:
⍝day15_2 data15  ⍝ takes a while...

The search on the full 500x500 matrix takes a while, but not enough to
think harder about this.

## Day 16

[Puzzle link](https://adventofcode.com/2021/day/16)

Today we get to parse a nested binary serialization format from hex numbers.
The easiest representation is to use nested arrays so this isn't going to
be super APLy or anything.

In [436]:
hex_to_binary←{⊃,/(2 2 2 2⊤⊢)¨(¯1+(⎕D,⎕A)⍳⊢)⍵}

In [455]:
⎕←data16←⊃⎕NGET 'day16-input.txt'

805311100469800804A3E488ACC0B10055D8009548874F65665AD42F60073E7338E7E5C538D8201
 

 14AEA1A19927797976F8F43CD7354D66747B3005B401397C6CBA2FCEEE7AACDECC017938B
 

 3F802E000854488F70FC401F8BD09E199005B3600BCBFEEE12FFBB84FC8466B515E92B79B
 

 1003C797AEBAF53917E99FF2E953D0D284359CA0CB80193D12B3005B4017968D77EB224B4
 

 6BBF591E7BEBD2FA00100622B4ED64773D0CF7816600B68020000874718E715C0010D8AF1
 

 E61CC946FB99FC2C20098275EBC0109FA14CAEDC20EB8033389531AAB14C72162492DE33A
 

 E0118012C05EEB801C0054F880102007A01192C040E100ED20035DA8018402BE20099A002
 

 0CB801AE0049801E800DD10021E4002DC7D30046C0160004323E42C8EA200DC5A87D06250
 

 C50015097FB2CFC93A101006F532EB600849634912799EF7BF609270D0802B59876F00424
 

 6941091A5040402C9BD4DF654967BFDE4A6432769CED4EC3C4F04C000A895B8E98013246A
 

 6016CB3CCC94C9144A03CFAB9002033E7B24A24016DD802933AFAE48EAA3335A632013BC4
 

 01D8850863A8803D1C61447A00042E3647B83F313674009E6533E158C3351F94C9902803D
 

 35C869865D564690103004E74CB001F39BEFFAAD37DFF558C012D005A5A9E851D25F76DD8
 

 8A5F4BC600ACB6E1322B004E5FE1F2FF0E3005EC017969EB7AE4D1A53D07B918C0B1802F0
 

 88B2C810326215CCBB6BC140C0149EE87780233E0D298C33B008C52763C9C94BF8DC88650
 

 4E1ECD4E75C7E4EA00284180371362C44320043E2EC258F24008747785D10C001039F8064
 

 4F201217401500043A2244B8D200085C3F8690BA78F08018394079A7A996D200806647A49
 

 E249C675C0802609D66B004658BA7F1562500366279CCBEB2600ACCA6D802C00085C658BD
 

 1DC401A8EB136100

Now we have to write a bunch of packet parsers operating on bit arrays.

In [442]:
]dinput
parse_packet←{
  ⍝ split off 3 bit version, 3 bit type, and remaining bits
  version type bits←1 0 0 1 0 0 1⊂⍵
  ⍝ dispatch to parsers for types given in part 1
  type≡1 0 0:parse_literal version bits
  length_type←1↑bits
  ⍝ operator length_type 0: 15-bit subpacket length in bits
  length_type=0:parse_operator_length version type(2⊥(15↑(1↓bits)))(16↓bits)
  ⍝ operator length_type 1: 11-bit subpacket count
  parse_operator_count version type(2⊥(11↑(1↓bits)))(12↓bits)
}

Literals don't nest and use a simple group encoding. We return the total length
because they go inside other packets that need to skip over them after parsing.

In [438]:
]dinput
parse_literal←{
  ⍝ returns (#bits ('literal' version value))
  version bits←⍵
  ⍝ 5-bit groups up to one starting with 0
  length←5×(((⍴bits)⍴1 0 0 0 0)/bits)⍳0
  value←(length⍴0 1 1 1 1)/length↑bits
  ((6+length)('literal'(2⊥version)(2⊥value)))
}

Length operators slurp up to length bits of nested subpackets.

In [439]:
]dinput
parse_operator_length←{
  ⍝ returns (#bits ('operator' version type subpackets))
  version type length bits←⍵
  subpackets←{
    ⍵=length:⍬
    sublength packet←parse_packet ⍵↓bits
    (⊂packet),(∇(sublength+⍵))
  }
  ((6+1+15+length)('operator'(2⊥version)(2⊥type)(subpackets 0)))
}

And count operators slurp up to count nested subpackets.

In [440]:
]dinput
parse_operator_count←{
  ⍝ returns (#bits ('operator' version type subpackets))
  version type count bits←⍵
  length←0
  subpackets←{
    0=⍵:⍬
    sublength packet←parse_packet length↓bits
    length+←sublength
    (⊂packet),(∇(⍵-1))
  }
  ((6+1+11+length)('operator'(2⊥version)(2⊥type)(subpackets count)))
}

### Part 1

We're supposed to add up "version" fields in each nested packet to prove
that we are parsing correctly I guess.

In [444]:
]dinput
version_sum←{
  version←2⌷⍵  ⍝ version is always at index 2
  (1⌷⍵)≡⊂'operator':version++/∇¨(⊃⍵[4])
  version
}

In [447]:
version_sum 2⊃parse_packet hex_to_binary'C0015000016115A2E0802F182340'

23

In [448]:
version_sum 2⊃parse_packet hex_to_binary'A0016C880162017C3686B18A3D4780'

31

In [449]:
version_sum 2⊃parse_packet hex_to_binary'620080001611562C8802118E34'

12

In [450]:
version_sum 2⊃parse_packet hex_to_binary'8A004A801A8002F478'

16

In [451]:
day16_1←{version_sum 2⊃parse_packet hex_to_binary ⍵}

In [456]:
day16_1 data16

971

### Part 2

Now we get to write an expression evaluator.

In [458]:
]dinput
evaluate←{
  ⍝ literals evaluate to fixed value
  (1⌷⍵)≡⊂'literal':3⌷⍵
  ⍝ everything else is an operator
  values←evaluate¨(⊃⍵[4])
  type←3⌷⍵
  type=0:+/values
  type=1:×/values
  type=2:⌊/values
  type=3:⌈/values
  type=5:>/values
  type=6:</values
  type=7:=/values
}

In [459]:
day16_2←{evaluate 2⊃parse_packet hex_to_binary ⍵}

In [460]:
day16_2¨'C200B40A82' '04005AC33890' '880086C3E88112' 'CE00C43D881120' 'D8005AC2A8F0' 'F600BC2D8F' '9C005AC2F8F0' '9C0141080250320F1802104A08'

3 54 7 9 1 0 0 1

The examples seem to work ok. This is pretty convenient and we
could probably represent the tree as an APL expression instead, and
use ⍎. Not for the first time I'm wondering if they had APL on the
brain this year...

In [461]:
day16_2 data16

831996589851

## Day 17

[Puzzle link](https://adventofcode.com/2021/day/17)

Today the input is just 4 numbers so I'm going to type it in manually, being careful to use the unary ¯ instead of -.

In [513]:
test17←20 30 ¯10 ¯5  ⍝ xmin..xmax ymin..ymax

In [514]:
data17←240 292 ¯90 ¯57

We are doing something with trajectories, which we will represent as
2d arrays with one row per step. The current state is a 4-tuple x y vx vy
in the last row that updates like this:

In [522]:
step_projectile←{x y vx vy←,¯1↑⍵ ⋄ ⍵ ⍪ ((x+vx) (y+vy) (vx-×vx) (vy-1))}

We'll iterate this until we've flown past the target. Since vy decreases
monotonically and the target is below the x-axis, we'll just say that
happens when y < ymin.

In [525]:
]dinput
trajectory←{
  vx vy←⍺
  xmin xmax ymin ymax←⍵
  past_target←{_ y _ _←,¯1↑⍵ ⋄ (y<ymin)}
  (step_projectile⍣past_target)1 4⍴0 0 vx vy
}

In [526]:
7 2 trajectory 20 30 ¯10 ¯5

0 0 7 2
 7 2 6 1
13 3 5 0
18 3 4 ¯1
22 2 3 ¯2
25 0 2 ¯3
27 ¯3 1 ¯4
28 ¯7 0 ¯5
28 ¯12 0 ¯6
28 ¯18 0 ¯7

### Part 1

We'll just search velocities to find the best one. The target is to the
right so we want 0 < vx ≤ xmax. Also ymin ≤ vy since any lower would go
past immediately, and vy ≤ xmax since we get at most xmax x steps and vy
only decreases by 1 each step.

In [532]:
]dinput
day17_1←{
  xmin xmax ymin ymax←⍵
  ⍝ returns 1 if point is in target, else 0
  in_target←{
    x y _ _←⍵ ⋄ (x≥xmin)∧(x≤xmax)∧(y≥ymin)∧(y≤ymax)
  }
  ⍝ tries trajectory vx vy, returns max height if hits target else 0
  trial←{
    vx vy←⍵
    points←vx vy trajectory xmin xmax ymin ymax
    hit←∨/(in_target⍤1)points
    hit:2⌷⌈⌿points
    0
  }
  ⌈/,trial¨⍳xmax xmax  ⍝ not quite enough range but works for part 1
}

In [530]:
day17_1 test17

45

In [531]:
⍝ day17_1 data17  ⍝ embarrassingly slow

4005

### Part 2

Now we want all the velocities. This is sorta simpler actually...

In [534]:
]dinput
day17_2←{
  xmin xmax ymin ymax←⍵
  ⍝ returns 1 if point is in target, else 0
  in_target←{
    x y _ _←⍵ ⋄ (x≥xmin)∧(x≤xmax)∧(y≥ymin)∧(y≤ymax)
  }
  ⍝ returns 1 if trajectory with initial velocity vx vy hits target else 0
  trial←{
    vx vy←⍵
    points←vx vy trajectory xmin xmax ymin ymax
    ∨/(in_target⍤1)points
  }
  +/,trial¨(⍳xmax(1+xmax+|ymin))-⊂0(|ymin-1)
}

In [535]:
day17_2 test17

112

In [536]:
day17_2 data17

2953

## Day 18

[Puzzle link](https://adventofcode.com/2021/day/18)

We get to do arithmetic with "snailfish numbers" aka nested pairs.
This involves a bunch of scanning "left" and "right" so it's going to
be simpler just to treat them as strings than to bother parsing.
I did try, but gave up when Dyalog crashed with error 139 or something
while I was trying to debug a basic tree traversal - you get what you
pay for, I guess.

In [564]:
test18←⊃⎕NGET 'day18-test.txt' 1

In [555]:
data18←⊃⎕NGET 'day18-input.txt' 1

Adding a and b is just concatenating

In [615]:
add_snailfish←{'[',⍺,',',⍵,']'}

In [616]:
'[1,2]' add_snailfish '[[3,4],5]'

[[1,2],[[3,4],5]]

But then reducing involves a bunch of hoo-ha.

In [562]:
]dinput
explode←{
  start←((+\(⍵='[')-(⍵=']'))=5)⍳1  ⍝ start of first pair to explode
  start>≢⍵:⍵                       ⍝ if none, no need to explode
  len←(start↓⍵)⍳']'                ⍝ end of first pair
  a b←⍎(len-1)↑(start↓⍵)           ⍝ get numbers a b in pair
  ⍝ add a to rightmost in left substring
  add_to_left←{
    (⍕(a+⍎(⍵.Lengths[2]↑⍵.Match))),(⍵.Lengths[2]↓⍵.Match)
  }
  ⍝ add b to leftmost in right substring
  add_to_right←{
    (⍵.Offsets[2]↑⍵.Match),⍕(b+⍎(⍵.Offsets[2]↓⍵.Match))
  }
  ⍝ form new left, right substrings
  left←('(\d+)[^\d]*$'⎕R add_to_left)⊢(start-1)↑⍵
  right←('^[^\d]*(\d+)'⎕R add_to_right)⊢(start+len)↓⍵
  left,'0',right
}

In [566]:
(explode '[[[[[9,8],1],2],3],4]')≡'[[[[0,9],2],3],4]'

1

In [567]:
(explode '[7,[6,[5,[4,[3,2]]]]]')≡'[7,[6,[5,[7,0]]]]'

1

In [568]:
(explode '[[6,[5,[4,[3,2]]]],1]')≡'[[6,[5,[7,0]]],3]'

1

In [569]:
(explode '[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')≡'[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]'

1

In [570]:
(explode '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]')≡'[[3,[2,[8,0]]],[9,[5,[7,0]]]]'

1

The other hoo-ha operation in reducing is "split".

In [586]:
split←'(\d\d+).*'⎕R{n←⍎⍵.Lengths[2]↑⍵.Match ⋄ '[',(⍕⌊n÷2),',',(⍕⌈n÷2),']',⍵.Lengths[2]↓⍵.Match}

In [587]:
split '10'

[5,5]

In [573]:
split '11'

[5,6]

In [575]:
(split '[[[[0,7],4],[15,[0,13]]],[1,1]]')≡'[[[[0,7],4],[[7,8],[0,13]]],[1,1]]'

1

In [576]:
(split '[[[[0,7],4],[[7,8],[0,13]]],[1,1]]')≡'[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]'

1

To reduce, iterate explode and split.

In [588]:
reduce←(split∘(explode⍣≡))⍣≡

In [589]:
reduce '[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]'

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

Adding is folding (left) then reducing. APL's fold is right-to-left
(because of course) so we need a helper for fold left. Note we have
to be careful to reduce after each addition because otherwise we might
end up with pairs nested more than 4 deep which breaks assumptions
in explode.

In [638]:
add_reduce←(reduce add_snailfish){↑⍺⍺⍨/(⌽⍵),⍬}⊢

In [639]:
(add_reduce '[1,1]' '[2,2]' '[3,3]' '[4,4]')≡'[[[[1,1],[2,2]],[3,3]],[4,4]]'

1

In [640]:
(add_reduce '[1,1]' '[2,2]' '[3,3]' '[4,4]' '[5,5]')≡'[[[[3,0],[5,3]],[4,4]],[5,5]]'

1

In [642]:
(add_reduce '[1,1]' '[2,2]' '[3,3]' '[4,4]' '[5,5]' '[6,6]')≡'[[[[5,0],[7,4]],[5,5]],[6,6]]'

1

In [645]:
larger_example←'[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]' '[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]' '[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]' '[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]' '[7,[5,[[3,8],[1,4]]]]' '[[2,[2,2]],[8,[8,1]]]' '[2,9]' '[1,[[[9,3],9],[[9,0],[0,7]]]]' '[[[5,[7,4]],7],1]' '[[[[4,2],2],6],[8,7]]'

In [646]:
(add_reduce larger_example)≡'[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]'

1

### Part 1

We're supposed to multiply and add up the tree of pairs. If APL had operator
precedence, we could substute +2× for ',' and so on and use ⍎. But it doesn't,
so we have to build our own evaluator.

In [647]:
translate←{⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵}

In [650]:
parse_snailfish←⍎'[],' translate '() '

In [651]:
parse_snailfish '[[9,1],[1,9]]'

┌───┬───┐
│9 1│1 9│
└───┴───┘

In [655]:
]dinput
magnitude←{
  0=≡⍵: ⍵
  left right←⍵
  (3×(magnitude left))+(2×(magnitude right))
}

In [664]:
129=magnitude(parse_snailfish '[[9,1],[1,9]]')

1

In [665]:
143=magnitude(parse_snailfish '[[1,2],[[3,4],5]]')

1

In [666]:
1384=magnitude(parse_snailfish '[[[[0,7],4],[[7,8],[6,0]]],[8,1]]')

1

In [667]:
445=magnitude(parse_snailfish '[[[[1,1],[2,2]],[3,3]],[4,4]]')

1

In [669]:
791=magnitude(parse_snailfish '[[[[3,0],[5,3]],[4,4]],[5,5]]')

1

In [670]:
1137=magnitude(parse_snailfish '[[[[5,0],[7,4]],[5,5]],[6,6]]')

1

In [671]:
3488=magnitude(parse_snailfish '[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]')

1

Great, that all seems to work.

In [676]:
day18_1←magnitude(parse_snailfish (add_reduce ⊢))

In [677]:
day18_1 test18

4140

In [678]:
day18_1 data18

3216

### Part 2

Ok, find the largest magnitude for pairs.

In [688]:
day18_2←⌈/,∘(∘.{day18_1 ⍺ ⍵}⍨⊢)

In [689]:
day18_2 test18

3993

In [690]:
day18_2 data18

4643

## Day 19

[Puzzle link](https://adventofcode.com/2021/day/19)

Today we've got a bunch of cattywampus 3d scanners that separately measure
where some beacons are. We are gonna do a bunch of 3d transformations
so will want to turn the points into homogeneous coordinates.

All the "each" dots are super ugly today - I am sure there is a better way.
We have to be careful reading input because there is an extra newline
after each scanner's data except the last.

In [735]:
test19←{⍵,1}¨¨⍎¨¨¨','(≠⊆⊢)¨¨{{''≡⊃(¯1↑⍵): ¯1↓⍵ ⋄ ⍵}(1↓⍵)}¨{('s'∊¨⍵)⊂⍵}⊃⎕NGET'day19-test.txt' 1

In [736]:
data19←{⍵,1}¨¨⍎¨¨¨','(≠⊆⊢)¨¨{{''≡⊃(¯1↑⍵): ¯1↓⍵ ⋄ ⍵}(1↓⍵)}¨{('s'∊¨⍵)⊂⍵}⊃⎕NGET'day19-input.txt' 1

We will need to line some of the scanners' coordinate frames up.
Each of 3 axes might be negated or not and there are 6 permutations
of axes. But half of those transformations turn the universe inside
out, so det M=1 filters out the 24 we want.

In [737]:
permutations←{(⍳⍵)(,[⍳2](⊢,⍤1 0~)⍤1)⍣⍵⍉⍪⍬}

In [738]:
det←{-/+/×/[2](2 3⍴0 1 2 0 2 1)⌽↑⍵ ⍵}

In [739]:
orientations←{(1=(det⍤2)⍵)⌿⍵}↑↑{×⍵×(1-|⍵)⌽↑(1 0 0)}¨¨,(↓⍉(1-2×(2 2 2⊤⍳8)))∘.×↓(permutations 3)

In [740]:
orientations←↑{((↑↑⍵),0)⍪(0 0 0 1)}¨↓↓orientations  ⍝ turn 3x3s into 4x4s

We'll want the 4x4 identity matrix I4, and a helper to make
transformation matrices combining a translation and a rotation.

In [741]:
I4←↑(1 0 0 0)(0 1 0 0)(0 0 1 0)(0 0 0 1)

In [742]:
transform←{p rmat←⍵ ⋄ ((¯1↓(↑p))@(1 4)(2 4)(3 4))⊢rmat}  ⍝ note ignore 4th of p

To align two scanners, we guess which beacons i j are the same and
get points relative to that, also trying all rotations k of the second
scanner relative to the first.

In [743]:
]dinput
align←{
  scanner1 scanner2←⍵
  trial←{
    ⍝ count i j k up to corresponding array bounds
    none k j i←(2(≢orientations)(≢⊃scanner2)(≢⊃scanner1)⊤⍵)+(0 1 1 1)
    none:0  ⍝ no alignment possible
    ⍝ u translates scanner1 points relative to point i
    u←transform(-i⌷⊃scanner1)I4
    ⍝ v translates scanner2 points relative to point j and rotates
    ⍝ (note that point j must be rotated first)
    pj←⍉k⌷orientations+.×(⍉↑j⌷⊃scanner2)
    v←transform(-pj)(k⌷orientations)
    scanner1_p←↓↑{⍉u+.×(⍉↑⍵)}¨scanner1
    scanner2_p←↓↑{⍉v+.×(⍉↑⍵)}¨scanner2
    ⍝ count transformed point matches
    num_matches←scanner1_p(+/∊)scanner2_p
    num_matches≥12:(⌹u)+.×(v)  ⍝ transform from 2->1 frame
    ∇(⍵+1)
  }
  trial 0
}

In [730]:
align (1⌷test19) (2⌷test19)

¯1 0 0 68
 0 1 0 ¯1246
 0 0 ¯1 ¯43
 0 0 0 1

align returns a transformation matrix that maps back to scanner1's frame.
For example, the 10th beacon in scanner1's list is the same as the 1st
beacon in scanner2's list, so multiplying the result matrix canonicalizes
to scanner1's frame.

In [731]:
10⌷⊃(1⌷test19)

┌────────────────┐
│¯618 ¯824 ¯621 1│
└────────────────┘

In [732]:
⍉((align (1⌷test19) (2⌷test19)))+.×(⍉↑(1⌷⊃(2⌷test19)))

¯618 ¯824 ¯621 1

Then we can just search to figure out the relative positions of all the probes.

In [750]:
]dinput
align_all←{
  scanners←⍵
  done←1⍴1              ⍝ indices that are done
  todo←1↓⍳≢scanners     ⍝ indices left todo
  mats←{I4}¨⍳≢scanners  ⍝ transform to canonical frame
  ⍝ match a scanner from todo with one from done
  find_pair←{
    ⍝ iterate i j over todo and done
    none j i←(2(≢done)(≢todo)⊤⍵)+(0 1 1)
    none:'error'
    ith jth←(i⌷todo)(j⌷done)
    ⍝ try aligning scanner i with j
    r←align(jth⌷scanners)(ith⌷scanners)
    r≡0:∇(⍵+1)  ⍝ no alignment, iterate
    ⍝ update transform for scanner i
    mats[ith]←⊂(⊃mats[jth])+.×r
    ⍝ mark scanner i done
    done,←todo[i]
    todo⊢←((i-1)↑todo),(i↓todo)
    search 0
  }
  ⍝ search until all scanners are done
  search←{
    0=≢todo:mats
    find_pair 0
  }
  search 0
}

### Part 1

How many beacons are there total? We can just find all the transforms,
and then collect unique positions.

In [745]:
]dinput
day19_1←{
  scanners←⍵
  mats←align_all scanners
  ≢∪⊃,/{
    r←⊃mats[⍵]
    beacons←⊃scanners[⍵]
    ↓↑{⍉r+.×(⍉↑⍵)}¨beacons
  }¨⍳≢⍵
}

In [751]:
day19_1 test19

79

In [752]:
⍝day19_1 data19  ⍝ this is extremely... slow...

### Part 2

Find the max distance between two scanners. We can get this from their
transformation matrices.

In [754]:
]dinput
day19_2←{
  scanners←⍵
  mats←align_all scanners
  origins←{⍵[(1 4) (2 4) (3 4)]}¨mats
  ⌈/,∘.{+/|⍺-⍵}⍨origins
}

In [755]:
day19_2 test19

3621

In [756]:
⍝day19_2 data19  ⍝ very... very... slow

## Day 20

[Puzzle link](https://adventofcode.com/2021/day/20)

Ok, this one seems made for APL.

In [777]:
test20_lookup←'#'∘=¨⊃⊃⎕NGET 'day20-test.txt' 1

In [758]:
⎕←test20_bitmap←↑'#'∘=¨2↓⊃⎕NGET 'day20-test.txt' 1

1 0 0 1 0
1 0 0 0 0
1 1 0 0 1
0 0 1 0 0
0 0 1 1 1

In [760]:
data20_lookup←'#'∘=¨⊃⊃⎕NGET 'day20-input.txt' 1

In [761]:
data20_bitmap←↑'#'∘=¨2↓⊃⎕NGET 'day20-input.txt' 1

To enhance, we could just use the stencil operator with its default padding.
But trickly the input lookup maps index 0 to 1, so the array should be
1-padded on odd iterations!

In [772]:
enhance_even←{k←⍺ ⋄ ({k[1+2⊥,⍵]}⌺3 3⊢)0(,∘⌽∘⍉⍣20)⍵}

In [773]:
enhance_odd←{k←⍺ ⋄ ¯4↓[2]¯4↓4↓4↓[2]({k[1+2⊥,⍵]}⌺3 3⊢)k[1](,∘⌽∘⍉⍣20)⍵}

### Part 1

Count 1s after enhancing twice (after once would be infinite!)

In [774]:
day20_1←{+/,(⍺ enhance_odd (⍺ enhance_even ⍵))}

In [775]:
test20_lookup day20_1 test20_bitmap

35

In [776]:
data20_lookup day20_1 data20_bitmap

5275

### Part 2

Now enhance 50 times instead.

In [782]:
day20_2←{k←⍺ ⋄ +/,({(k enhance_odd (k enhance_even ⍵))}⍣25)⍵}

In [783]:
test20_lookup day20_2 test20_bitmap

3351

In [785]:
data20_lookup day20_2 data20_bitmap

16482

## Day 21

[Puzzle link](https://adventofcode.com/2021/day/21)

We get to play with dice today. My first instinct was to pass a dice
closure to a simulation function, but APL doesn't have first class functions.
So instead, I'll think about the sequences of rolls for each player.

In [867]:
(6{⍵⌿⍨0=⍺|⍳≢⍵}3+/1+100|⊢)((⍳306)-6)  ⍝ player 1

6 24 42 60 78 96 114 132 150 168 186 204 222 240 258 276 294 12 30 48 66 84 102

 120 138 156 174 192 210 228 246 264 282 200 18 36 54 72 90 108 126 144 1

 62 180 198 216 234 252 270 288

This looks like just 6+18n (mod 300) where n is the turn.

In [876]:
6+(300|(18×(⍳50)-1))

6 24 42 60 78 96 114 132 150 168 186 204 222 240 258 276 294 12 30 48 66 84 102

 120 138 156 174 192 210 228 246 264 282 300 18 36 54 72 90 108 126 144 1

 62 180 198 216 234 252 270 288

To find score totals we could add that up mod 10 and add the start position.

In [877]:
+\4+10|+\(6+(100|(18×(⍳50)-1)))  ⍝ 4 is start position

10 14 20 26 30 40 44 50 56 60 70 74 80 86 90 100 104 110 116 120 130 134 140 14

 6 150 160 164 170 176 180 190 194 200 206 210 220 224 230 236 240 250 254

 260 266 270 280 284 290 296 300

But note that positions are cyclical with period 5 for player 1 and
10 for player 2, hmm. This is gonna be a little overcomplicated for
part 1, but why not.

In [909]:
10(⊢+⊣×0=⊢)10|4++\(6+(100|(18×(⍳50)-1)))  ⍝ player 1

10 4 6 6 4 10 4 6 6 4 10 4 6 6 4 10 4 6 6 4 10 4 6 6 4 10 4 6 6 4 10 4 6 6 4 10

 4 6 6 4 10 4 6 6 4 10 4 6 6 4

In [910]:
10(⊢+⊣×0=⊢)10|8++\(15+(100|(18×(⍳50)-1)))  ⍝ player 2

3 6 7 6 3 8 1 2 1 8 3 6 7 6 3 8 1 2 1 8 3 6 7 6 3 8 1 2 1 8 3 6 7 6 3 8 1 2 1 8

 3 6 7 6 3 8 1 2 1 8

I bet all the positions repeat every 10 turns.

In [936]:
ten_turns←{start offset←⍵ ⋄ 10(⊢+⊣×0=⊢)10|start++\(offset+(100|(18×(⍳10)-1)))}

In [937]:
ten_turns 4 6

10 4 6 6 4 10 4 6 6 4

In [938]:
ten_turns 6 15

1 4 5 4 1 6 9 10 9 6

This suggests a way to compute the score for turn number n.

In [939]:
score←{seq n←⍵ ⋄ (+\(0,seq))[1+(10|n)]+((⌊(n÷10))×(+/seq))}

In [948]:
score (ten_turns 4 6) 166

1000

In [949]:
¯1↑+\4+10|+\(6+(100|(18×(⍳166)-1)))

1000

### Part 1

Find the winning turn.  We can just binary search... but I really
don't want to write binary search.

TIL that Dyalog has a
[standard library](https://aplwiki.com/wiki/Dfns_workspace)!
You can copy stuff out of the baked in 'dfns' workspace... that's
kinda nice. Note that binary search takes a function as an
argument by actually being an "operator", which is allowed to
operate on functions.

In [953]:
'bsearch'⎕CY'dfns'

In [963]:
]dinput
day21_1←{
  start1 start2←⍵
  score1←{(score (ten_turns start1 6) ⍵)}
  score2←{(score (ten_turns start2 15) ⍵)}
  max_turns←1e12
  win1←{(score1 ⍵)≥1000} bsearch 1 max_turns
  win2←{(score2 ⍵)≥1000} bsearch 1 max_turns
  win1<win2: ((6×(win1-1))+3)×(score2 (win1-1))  ⍝ p1 wins
  (6×win2)×(score1 win2) ⍝ p2 wins
}

In [964]:
day21_1 4 8

739785

In [965]:
day21_1 8 1

518418

### Part 2

Ok, now we have a quantum die, and have to count winning games.
This seems like a dynamic programming thing. We will tabulate
outcomes (#p1wins #p2wins), filling in backwards to avoid crazy
recursion.

I am going to use for loops, because it is easier to understand.

In [972]:
dirac_table←2 10 21 10 21⍴(0 0)  ⍝ to_play p1 p1_score+1 p2 p2_score+1

In [973]:
 ]dinput
 r←tabulate_dirac_wins;to_play;p1;p1_n;p2;p2_n;p1_score;p2_score;r1;r2;r3;outcome
 mod10←{10(⊢+⊣×0=⊢)(10|⍵)}
 :For p1_score :In ⌽(⍳21)-1
     :For p2_score :In ⌽(⍳21)-1
         :For p1 :In ⍳10
             :For p2 :In ⍳10
                 :For to_play :In ⍳2
                     :For r1 :In ⍳3
                         :For r2 :In ⍳3
                             :For r3 :In ⍳3
                                 :If to_play=1
                                     p1_n←mod10(p1+r1+r2+r3)
                                     :If (p1_score+p1_n)≥21
                                         outcome←⊂1 0
                                     :Else
                                         outcome←dirac_table[2;p1_n;1+p1_score+p1_n;p2;1+p2_score]
                                     :EndIf
                                 :Else ⍝ to_play=2
                                     p2_n←mod10(p2+r1+r2+r3)
                                     :If (p2_score+p2_n)≥21
                                         outcome←⊂0 1
                                     :Else
                                         outcome←dirac_table[1;p1;1+p1_score;p2_n;1+p2_score+p2_n]
                                     :EndIf
                                 :EndIf
                                 dirac_table[to_play;p1;1+p1_score;p2;1+p2_score]+←outcome
                             :EndFor
                         :EndFor
                     :EndFor
                 :EndFor
             :EndFor
         :EndFor
     :EndFor
 :EndFor

In [974]:
tabulate_dirac_wins

In [983]:
day21_2←{start1 start2←⍵ ⋄ ⌈/⊃dirac_table[1;start1;1;start2;1]}

In [981]:
⎕PP←16  ⍝ big numbers

In [984]:
day21_2 4 8

444356092776315

In [985]:
day21_2 8 1

116741133558209

## Day 22

[Puzzle link](https://adventofcode.com/2021/day/22)

3d bitmap wrangling today.

In [2]:
]dinput
parse_cuboid←{
  ⍝ extract on/off and numbers from 'on x=-20..26,y=-36..17,z=-47..7'
  ⍝ return 0|1 xmin xmax ymin ymax zmin zmax
  xs←' '(≠⊆⊢)('(on|off) x=(-?\d+)..(-?\d+),y=(-?\d+)..(-?\d+),z=(-?\d+)..(-?\d+)'⎕R'\1 \2 \3 \4 \5 \6 \7')⊢⍵
  ((⊃(1↑xs))≡'on'),(⍎¨1↓xs)
}

In [4]:
⎕←test22←parse_cuboid¨⊃⎕NGET 'day22-test.txt' 1

┌─────────────────────┬──────────────────────┬──────────────────────┬──────────
│1 ¯20 26 ¯36 17 ¯47 7│1 ¯20 33 ¯21 23 ¯26 28│1 ¯22 28 ¯29 23 ¯38 16│1 ¯46 7 ¯6
└─────────────────────┴──────────────────────┴──────────────────────┴──────────

 ──────────┬────────────────────┬────────────────────┬────────────────────
 46 ¯50 ¯1│1 ¯49 1 ¯3 46 ¯24 28│1 2 47 ¯22 22 ¯23 27│1 ¯27 23 ¯28 26 ¯21 
 ──────────┴────────────────────┴────────────────────┴────────────────────

 ──┬───────────────────┬─────────────────────┬──────────────────────┬─────
 29│1 ¯39 5 ¯6 47 ¯3 44│1 ¯30 21 ¯8 43 ¯13 34│1 ¯22 26 ¯27 20 ¯29 19│0 ¯48
 ──┴───────────────────┴─────────────────────┴──────────────────────┴─────

 ──────────────────┬────────────────────┬────────────────────────┬────────
 ¯32 26 41 ¯47 ¯37│1 ¯12 35 6 50 ¯50 ¯2│0 ¯48 ¯32 ¯32 ¯16 ¯15 ¯5│1 ¯18 26
 ──────────────────┴────────────────────┴────────────────────────┴────────

 ─────────────┬───────────────────────┬─────────────────────┬─────────────
 ¯33 15 ¯7 46│0 ¯40 ¯22 ¯38 ¯28 23 41│1 ¯16 35 ¯41 10 ¯47 6│0 ¯32 ¯23 11 
 ─────────────┴───────────────────────┴─────────────────────┴─────────────

 ────────┬─────────────────────┬────────────────────┬────────────────────┬
 30 ¯14 3│1 ¯49 ¯5 ¯3 45 ¯29 18│0 18 30 ¯20 ¯8 ¯3 13│1 ¯41 9 ¯7 43 ¯33 15│
 ────────┴─────────────────────┴────────────────────┴────────────────────┴

 ─────────────────────────────────────────┬───────────────────────────────
 1 ¯54112 ¯39298 ¯85059 ¯49293 ¯27449 7877│1 967 23432 45373 81175 27513 5
 ─────────────────────────────────────────┴───────────────────────────────

 ────┐
 3682│
 ────┘

In [5]:
data22←parse_cuboid¨⊃⎕NGET 'day22-input.txt' 1

### Part 1

Count the "on" cubes in the volume ¯50≤x≤50, ¯50≤y≤50, ¯50≤z≤50.

In [13]:
]dinput
day22_1←{
  ⍝ we'll make a big 3d bitmap and just count
  cubes←101 101 101⍴0
  _←{
    bit x0 x1 y0 y1 z0 z1←⍵
    (x1<¯50)∨(x0>50):0
    (y1<¯50)∨(y0>50):0
    (z1<¯50)∨(z0>50):0
    cuboid←(⊂(50+x0)(50+y0)(50+z0))+(⍳((x1-x0)+1)((y1-y0)+1)((z1-z0)+1))
    cubes⊢←bit@cuboid⊢cubes
    0
  }¨⍵
  +/,cubes
}

In [7]:
day22_1 test22

590784

In [8]:
day22_1 data22

602574

### Part 2

Count all the "on" cubes.  We get a new test case.

In [12]:
test22_2←parse_cuboid¨⊃⎕NGET 'day22-test2.txt' 1

In [23]:
↑(⌊/test22_2),⌈/test22_2

0 ¯120100 ¯40997 ¯124565 ¯50804 ¯121762 ¯30197
1 55694 120875 50097 118853 54442 119054

The volume is about 250k^3 which is too big for a bitmap.
We could compute new cuboids for each cuboid-cuboid union or
intersection, and keep a running list. Or we could do the
same for rectangles on 250k slices. But I'd prefer to use a
subdivision algorithm instead if at all possible so I don't
have to think about all the weird cases.

We'll need to intersect cuboids with aribtrary volumes.

In [39]:
]dinput
intersect_cuboids_with_volume←{
  ⍝ all ranges are closed [x0,x1] [y0,y1] [z0,z1]
  vx0 vx1 vy0 vy1 vz0 vz1 cuboids←⍵
  rs←{
    bit x0 x1 y0 y1 z0 z1←⍵
    (x1<vx0)∨(x0>vx1): 0
    (y1<vy0)∨(y0>vy1): 0
    (z1<vz0)∨(z0>vz1): 0
    ⍝ max start, min end per axis
    xs←(⌈/x0 vx0) (⌊/x1 vx1)
    ys←(⌈/y0 vy0) (⌊/y1 vy1)
    zs←(⌈/z0 vz0) (⌊/z1 vz1)
    bit,xs,ys,zs
  }¨cuboids
  ⍝ filter only overlapping cuboids
  (0∘≢¨rs)/rs
}

In [40]:
intersect_cuboids_with_volume ¯10 10 ¯10 10 ¯10 10 (⊂0 0 5 0 5 0 5)

┌─────────────┐
│0 0 5 0 5 0 5│
└─────────────┘

In [41]:
intersect_cuboids_with_volume ¯10 10 ¯10 10 ¯10 10 ((0 0 5 0 5 0 5)(1 11 15 0 5 0 5)(1 9 15 0 5 0 5))

┌─────────────┬──────────────┐
│0 0 5 0 5 0 5│1 9 10 0 5 0 5│
└─────────────┴──────────────┘

To count cubes in a volume, consider the cuboids inside. If there are
none, there are 0 cubes. If there's one, there are however many cubes
that has. If there are two or more cuboids, take the last, and sum the
count of cubes outside (recursively) and inside. Since it is the last
cuboid from the list, it takes precedence, so we don't need to recurse
to count inside - the count is just its bit times its volume.

The recursion is error-prone and it took me a while to find some bugs
subdividing volumes! Stepping through simple examples was a big help.

In [45]:
]dinput
day22_2←{
  ⍝ number of on cubes in a cuboid
  on_cubes←{bit x0 x1 y0 y1 z0 z1←⍵ ⋄ bit×(1+(x1-x0))×(1+(y1-y0))×(1+(z1-z0))}
  count←{
    x0 x1 y0 y1 z0 z1 cs←⍵
    0=≢cs:0
    1=≢cs:on_cubes ⊃cs
    ⍝ if two or more cuboids, partition on the last
    bit cx0 cx1 cy0 cy1 cz0 cz1←⊃(¯1↑cs)
    cs←¯1↓cs
    ⍝ non-overlapping nested volumes outside the last cuboid
    left←x0(cx0-1)y0 y1 z0 z1        ⍝ <cx0
    right←(cx1+1)x1 y0 y1 z0 z1      ⍝ >cx1
    below←cx0 cx1 y0(cy0-1)z0 z1     ⍝ in [cx0,cx1], <cy0 
    above←cx0 cx1(cy1+1)y1 z0 z1     ⍝ in [cx0,cx1], >cy1 
    behind←cx0 cx1 cy0 cy1 z0(cz0-1) ⍝ in [cx0,cx1], in [cy0,cy1], <cz0
    front←cx0 cx1 cy0 cy1(cz1+1)z1   ⍝ in [cx0,cx1], in [cy0,cy1], >cz1
    ⍝ count cubes outside
    num_outside←+/{
      x0 x1 y0 y1 z0 z1←⍵
      (x0>x1)∨(y0>y1)∨(z0>z1): 0 ⍝ empty
      count ⍵,⊂(intersect_cuboids_with_volume ⍵,⊂cs)
    }¨(left right below above behind front)
    ⍝ as this is the last cuboid, it takes precedence over its volume
    num_outside+(on_cubes bit cx0 cx1 cy0 cy1 cz0 cz1)
  }
  ⍝ bound all the cuboids
  _ x0 _ y0 _ z0 _←⊃⌊/⍵
  _ _ x1 _ y1 _ z1←⊃⌈/⍵
  count x0 x1 y0 y1 z0 z1 ⍵
}

In [46]:
day22_2 (¯2↓test22) ⍝ same as part 1

590784

In [48]:
⎕PP←16

In [49]:
day22_2 test22_2

2758514936282235

In [50]:
day22_2 data22

1288707160324706

## Day 23

[Puzzle link](https://adventofcode.com/2021/day/23)

Another search problem today...

In [2]:
⎕←test23←↑⊃⎕NGET 'day23-test.txt' 1

#############
#...........#
###B#C#B#D###
 #A#D#C#A# 
 #########

In [3]:
data23←↑⊃⎕NGET 'day23-input.txt' 1

We are supposed to find the lowest cost way to move some letters
(amphipods, I'll call them pods) around a nethack level. We will
need a way to enumerate moves and their costs.

⍳ is irritating when you don't want 1..n, let's write a more generic
range function:

In [4]:
range←{a b←⍵ ⋄ a+((×(b-a))×((⍳1+|(b-a))-1))}

In [5]:
range 3 4

3 4

In [6]:
range 6 1

6 5 4 3 2 1

Here are the ways to move pods following the rules.

In [7]:
]dinput
enumerate_pod_moves←{
  grid←⍵
  ⍝ move if hallway clear
  move←{
    p from to←⍵
    ⍝ offset "from" so we don't count the square where the pod is
    ⍝ (if it's in a room, this ignores the doorway, but since doorways
    ⍝ are never occupied this is ok)
    dx←×(to[2]-from[2])
    ∨/'.'≠grid[{2,⍵}¨range(dx+from[2])(to[2])]:⍬ ⍝ blocked
    (p from to)
  }
  ⍝ move pod from "from" into its room, if allowed
  move_to_room←{
    p from room←⍵
    ⍝ ok if room is empty: take the deepest cell
    grid[room]≡'..':⊂move(p from (⊃room[2]))
    ⍝ ok if room has another p pod
    grid[room]≡('.',p):⊂move(p from (⊃room[1]))
    ⍬
  }
  ⍝ non-doorway hallway positions
  hallway←(2 2)(2 3)(2 5)(2 7)(2 9)(2 11)(2 12)
  ⍝ enumerate moves for each pod
  moves←↑,/{
    from←⍵
    p←grid[⊂from]
    ⍝ can only move from hallway to own room
    ((⊂from)∊hallway)∧(p='A'):move_to_room p from((3 4)(4 4))
    ((⊂from)∊hallway)∧(p='B'):move_to_room p from((3 6)(4 6))
    ((⊂from)∊hallway)∧(p='C'):move_to_room p from((3 8)(4 8))
    ((⊂from)∊hallway)∧(p='D'):move_to_room p from((3 10)(4 10))
    ⍝ else move to hallway
    grid[⊂from-1 0]≠'.':⍬ ⍝ blocked
    {move p from ⍵}¨hallway
  }¨grid(⍸∊)'ABCD'
  (({(≢⍵)>0}¨moves)/moves) ⍝ filter out empty moves
}

In [8]:
enumerate_pod_moves test23

┌───────────┬───────────┬───────────┬───────────┬───────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬─────────────┬─────────────┐
│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│┌─┬────┬───┐│┌─┬────┬───┐│┌─┬────┬───┐│┌─┬────┬───┐│┌─┬────┬───┐│┌─┬────┬────┐│┌─┬────┬────┐│
││B│3 4│2 2│││B│3 4│2 3│││B│3 4│2 5│││B│3 4│2 7│││B│3 4│2 9│││B│3 4│2 11│││B│3 4│2 12│││C│3 6│2 2│││C│3 6│2 3│││C│3 6│2 5│││C│3 6│2 7│││C│3 6│2 9│││C│3 6│2 11│││C│3 6│2 12│││B│3 8│2 2│││B│3 8│2 3│││B│3 8│2 5│││B│3 8│2 7│││B│3 8│2 9│││B│3 8│2 11│││B│3 8│2 12│││D│3 10│2 2│││D│3 10│2 3│││D│3 10│2 5│││D│3 10│2 7│││D│3 10│2 9│││D│3 10│2 11│││D│3 10│2 12││
│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│└─┴────┴───┘│└─┴────┴───┘│└─┴────┴───┘│└─┴────┴───┘│└─┴────┴───┘│└─┴────┴────┘│└─┴────┴────┘│
└───────────┴───────────┴───────────┴───────────┴───────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┴────────────┴────────────┴────────────┴────────────┴────────────┴────────────┴────────────┴─────────────┴─────────────┘

Let's check going into rooms.

In [9]:
enumerate_pod_moves ↑('#############')('#...B.......#')('###B#C#.#D###')('  #A#D#C#A#  ')('  #########  ')

┌───────────┬───────────┬───────────┬───────────┬────────────┬────────────┬────────────┬────────────┬─────────────┬─────────────┬───────────┬───────────┬────────────┬────────────┐
│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│┌─┬────┬───┐│┌─┬────┬───┐│┌─┬────┬────┐│┌─┬────┬────┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│
││B│3 4│2 2│││B│3 4│2 3│││C│3 6│2 7│││C│3 6│2 9│││C│3 6│2 11│││C│3 6│2 12│││D│3 10│2 7│││D│3 10│2 9│││D│3 10│2 11│││D│3 10│2 12│││C│4 8│2 7│││C│4 8│2 9│││C│4 8│2 11│││C│4 8│2 12││
│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│└─┴────┴───┘│└─┴────┴───┘│└─┴────┴────┘│└─┴────┴────┘│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│
└───────────┴───────────┴───────────┴───────────┴────────────┴────────────┴────────────┴────────────┴─────────────┴─────────────┴───────────┴───────────┴────────────┴────────────┘

Note that we treat C (3 6) exiting its starting room into the
hallway (2 7) and going to its destination room (3 8) as two
separate moves. Hopefully that won't burn too much search time.

In [10]:
enumerate_pod_moves ↑('#############')('#...B.C.....#')('###B#.#.#D###')('  #A#D#C#A#')('  #########')

┌───────────┬───────────┬───────────┬────────────┬─────────────┬─────────────┬───────────┬────────────┬────────────┐
│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬───┬───┐│┌─┬────┬───┐│┌─┬────┬────┐│┌─┬────┬────┐│┌─┬───┬───┐│┌─┬───┬────┐│┌─┬───┬────┐│
││C│2 7│3 8│││B│3 4│2 2│││B│3 4│2 3│││D│3 10│2 9│││D│3 10│2 11│││D│3 10│2 12│││C│4 8│2 9│││C│4 8│2 11│││C│4 8│2 12││
│└─┴───┴───┘│└─┴───┴───┘│└─┴───┴───┘│└─┴────┴───┘│└─┴────┴────┘│└─┴────┴────┘│└─┴───┴───┘│└─┴───┴────┘│└─┴───┴────┘│
└───────────┴───────────┴───────────┴────────────┴─────────────┴─────────────┴───────────┴────────────┴────────────┘

Of course, we have to be able to update the nethack grid.

In [11]:
do_move←{move grid←⍵ ⋄ p from to←move ⋄ ('.'p@from to)⊢grid}

In [12]:
do_move (⊃enumerate_pod_moves test23) test23

#############
#B..........#
###.#C#B#D###
 #A#D#C#A# 
 #########

### Part 1

Find the best cost to sort the pods into the correct rooms.

A best-first search was way too slow. Remembering [day 12](#day-12),
first I tried never dropping things from the search queue. Then I
tried A* with a simple heuristic: each letter must move to the
correct x position, so it'll cost at least its movement cost
times its x distance from its room. This was still way too slow,
so I wrote it up in Python. That took 30 minutes to code and
debug and 1 minute to run on test23 with a heapq, but was still
too slow on the part 1 input.

D moves are so expensive that you have to search the entire space before
you make the needful D moves, so we need a better heuristic. Adding
+1 or +2 to move out of the starting room, plus +1 to move into
the correct room sped the Python search up to 11s for test23 and 48s
for data23. Some more optimizations got that down to ~1s and ~4s.

I wasn't having fun with this part anymore, so didn't bother to refine
the APL search program. Dyalog doesn't seem to have a heap library.
The program might still have some bugs, too. Maybe I will come back
later and improve it.

In [ ]:
]dinput
best_pod_cost←{
  grid←⍵
  ⍝ cost per move
  cost←{
      p from to←⍵ ⋄ steps←+/|from-to
      p='A': steps
      p='B': 10×steps
      p='C': 100×steps
      p='D': 1000×steps
  }
  ⍝ when are we done
  done←{grid[(3 4)(4 4)(3 6)(4 6)(3 8)(4 8)(3 10)(4 10)]≡'AABBCCDD'}
  ⍝ lower bound on cost remaining
  estimate_remaining_cost←{
    grid←⍵
    a_cost←+/{1×|2⊃⍵-4}¨(grid(⍸∊)'A')
    b_cost←+/{10×|2⊃⍵-6}¨(grid(⍸∊)'B')
    c_cost←+/{100×|2⊃⍵-8}¨(grid(⍸∊)'C')
    d_cost←+/{1000×|2⊃⍵-10}¨(grid(⍸∊)'D')
    a_cost+b_cost+c_cost+d_cost
  }
  ⍝ avoid some silly moves
  prune←{
      move grid←⍵ ⋄ p from to←move
      (p='A')∧(from≡(4 4)): 1
      (p='A')∧(from≡(3 4))∧(grid[⊂4 4]='A'): 1
      (p='B')∧(from≡(4 6)): 1
      (p='B')∧(from≡(3 6))∧(grid[⊂4 6]='A'): 1
      (p='C')∧(from≡(4 8)): 1
      (p='C')∧(from≡(3 8))∧(grid[⊂4 8]='A'): 1
      (p='D')∧(from≡(4 10)): 1
      (p='D')∧(from≡(3 10))∧(grid[⊂4 10]='A'): 1
      0
  }
  ⍝ best-first search with heuristic
  q←(,⊂(0 0 grid))
  search←{
    0=≢q:0
    i←(⊃⍋)q  ⍝ min sort score
    cur_sort cur_cost cur_grid←⊃q[i]
    done cur_grid:cur_cost ⍝ answer
    moves←enumerate_pod_moves cur_grid
    moves←({~(prune ⍵ cur_grid)}¨moves)/moves
    q[i]←⊂(100000000 cur_cost cur_grid)
    0=≢moves:∇ 0
    q,←{
      new_grid←do_move ⍵ cur_grid
      new_cost←cur_cost+(cost ⍵)
      new_sort←new_cost+(estimate_remaining_cost new_grid)
      (new_sort new_cost new_grid)
    }¨moves
    ∇ 0
  }
  search 0
}

### Part 2

Now the rooms are bigger. I generalized and then added some
simple optimizations to my Python program, e.g. not revisiting
visited states. I also improved the heuristic to include a
cost for correctly placed letters having to move out of the
way to let incorrectly placed letters behind them move out.

The search took about 5 hours for part 2. I set it running,
then went out and did stuff for Christmas. Probably there was
some cleverer way to do this problem by thinking more abstractly
about sequences of swaps or something, but since A* finished, I
felt done.  _Worst.  Roguelike.  Ever._

Here is the Python program, for completeness' sake.

```python
#!/usr/bin/env python3

from datetime import datetime
import heapq

test = [[c for c in line.strip('\n')]
        for line in open('day23-test.txt').readlines()]
data = [[c for c in line.strip('\n')]
        for line in open('day23-input.txt').readlines()]
test2 = [[c for c in line.strip('\n')]
         for line in open('day23-test2.txt').readlines()]
data2 = [[c for c in line.strip('\n')]
         for line in open('day23-input2.txt').readlines()]

room_column = {'A': 3, 'B': 5, 'C': 7, 'D': 9}
def dest_in_room(grid, p):
  i = room_column[p]
  for j in range(len(grid)-2, 1, -1):
    if grid[j][i] == '.':
      return (j,i)
    if grid[j][i] != p:
      return

def moves(grid):
  for j in range(len(grid)):
    for i in range(len(grid[j])):
      p = grid[j][i]
      if p not in 'ABCD': continue
      if j == 1:  # move from hallway into room
        dest = dest_in_room(grid, p)
        if dest and is_hallway_clear(grid, i, dest[1]):
          yield p, j, i, dest[0], dest[1]
      else:  # move from room into hallway if not blocked
        if grid[j-1][i] != '.': continue
        for dest in ((1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11)):
          if is_hallway_clear(grid, i, dest[1]):
            yield p, j, i, dest[0], dest[1]

def is_hallway_clear(grid, x0, x1):
  assert x0 != x1
  dx = 1 if x1 > x0 else -1
  x = x0
  while True:
    x += dx
    if grid[1][x] != '.': return False
    if x == x1: return True

def make_move(move, grid):
  p, from_y, from_x, to_y, to_x = move
  new_grid = [row[:] for row in grid]
  new_grid[from_y][from_x] = '.'
  new_grid[to_y][to_x] = p
  return new_grid

def print_grid(grid):
  for j in range(len(grid)):
    s = ''
    for i in range(len(grid[j])):
      s += grid[j][i]
    print(s)

def done(grid):
  for j in range(len(grid)-2, 1, -1):
    if grid[j][3] != 'A': return False
    if grid[j][5] != 'B': return False
    if grid[j][7] != 'C': return False
    if grid[j][9] != 'D': return False
  return True

cost_per_step = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

def cost(move):
  p, from_y, from_x, to_y, to_x = move
  steps = abs(from_x-to_x) + abs(from_y-to_y)
  return cost_per_step[p] * steps

def prune(grid, move):
  p, from_y, from_x, to_y, to_x = move
  i = room_column[p]
  return from_x == i and all(grid[j][i] == p for j in range(from_y+1, len(grid)-1))

def hashable(grid):
  return ''.join(''.join(row) for row in grid)

def search(grid):
  q = [(0, 0, grid)]
  visited = set()
  while q:
    cur_sort, cur_cost, cur_grid = heapq.heappop(q)
    visited.add(hashable(cur_grid))
    if done(cur_grid):
      return cur_cost
    for move in moves(cur_grid):
      if not prune(cur_grid, move):
        new_grid = make_move(move, cur_grid)
        if hashable(new_grid) in visited: continue
        new_cost = cur_cost + cost(move)
        new_sort = new_cost + estimate_remaining_cost(new_grid)
        heapq.heappush(q, (new_sort, new_cost, new_grid))
  return -1

def blocking(grid, p, j0, i):
  return any(grid[j][i] != p for j in range(j0+1, len(grid)-1))

def estimate_remaining_cost(grid):
  cost = 0
  for j in range(len(grid)):
    for i in range(len(grid[j])):
      p = grid[j][i]
      cost += (j+abs(i-3)        if p == 'A' and i != 3 else
               j                 if p == 'A' and i == 3 and blocking(grid, p, j, i) else
               10*(j+abs(i-5))   if p == 'B' and i != 5 else
               10*j              if p == 'B' and i == 5 and blocking(grid, p, j, i) else
               100*(j+abs(i-7))  if p == 'C' and i != 7 else
               100*j             if p == 'C' and i == 7 and blocking(grid, p, j, i) else
               1000*(j+abs(i-9)) if p == 'D' and i != 9 else
               1000*j            if p == 'D' and i == 9 and blocking(grid, p, j, i) else
               0)
  return cost

def time(fn):
  start = datetime.now()
  result = fn()
  end = datetime.now()
  print(f'{result}  (took {end-start})')

time(lambda: search(data2))
time(lambda: search(test2))
```

## Day 24

[Puzzle link](https://adventofcode.com/2021/day/24)

Interesting, an ALU today. I like emulation!

In [14]:
data24←' '(≠⊆⊢)¨⊃⎕NGET'day24-input.txt' 1

This one is pretty simple, so I guess we'll just mostly be
untangling what the program does. Perhaps we'll want to
decompile it into an expression tree or something - not even
sure we strictly need to run the program, but why not:

In [100]:
]dinput
run_program←{
  program inputs←⍵
  registers←0 0 0 0 ⍝ w x y z
  decode_register_index←{⍵='w':1 ⋄ ⍵='x':2 ⋄ ⍵='y':3 ⋄ ⍵='z':4}
  decode_register_or_immediate←{
    ∧/⍵∊'wxyz':registers[decode_register_index ⍵] ⋄ ⍎⍵
  }
  run_instruction←{
    ⍝⎕←⍵ registers inputs
    ⍝ inp has one argument
    (2=≢⍵)∧(⍵[1]≡⊂'inp'):{
      registers[decode_register_index ⍵]←1↑inputs
      inputs⊢←1↓inputs
      0
    }⊃(⍵[2])
    ⍝ all other instructions have two arguments
    opcode arg1 arg2←⍵
    a_index←decode_register_index arg1
    a←registers[a_index]
    b←decode_register_or_immediate arg2
    opcode≡'add':{registers[a_index]←a+b ⋄ 0}0
    opcode≡'mul':{registers[a_index]←a×b ⋄ 0}0
    ⍝ divide and truncate
    opcode≡'div':{registers[a_index]←a((××∘⌊|)÷)b ⋄ 0}0
    opcode≡'mod':{registers[a_index]←b|a ⋄ 0}0
    opcode≡'eql':{registers[a_index]←a=b ⋄ 0}0
  }
  _←run_instruction¨program
  registers
}

In [17]:
⎕PP←16

In [101]:
run_program data24 (1 3 5 7 9 2 4 6 8 9 9 9 9 9)

9 1 22 2695331544

### Part 1

We have to find the largest input string such that the program
computes z=0.

Running on a couple random inputs doesn't give much insight...
z is just some 10-digit number. It seems to vary with the size
of the input number, and mostly increases as we read more input
digits. There are lots of repeated sequences of instructions,
so let's look at the program, per digit.

In [59]:
{⊃,/' '(1↓∘,,⍤0)⍵}¨⍉↑({('inp' (,'w'))≡⍵}¨data24)⊂data24

┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬─────────┬────────┬────────┬────────┬────────┬─────────┐
│inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │inp w │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │mul x 0 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │add x z │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mod x 26│mod x 26│mod x 26│mod x 26│mod x 26│mod x 26│mod x 26│mod x 26│mod x 26 │mod x 26│mod x 26│mod x 26│mod x 26│mod x 26 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│div z 1 │div z 1 │div z 1 │div z 26│div z 1 │div z 26│div z 1 │div z 26│div z 26 │div z 26│div z 1 │div z 1 │div z 26│div z 26 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add x 12│add x 11│add x 12│add x -3│add x 10│add x -9│add x 10│add x -7│add x -11│add x -4│add x 14│add x 11│add x -8│add x -10│
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │eql x w │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │eql x 0 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add y 25│add y 25│add y 25│add y 25│add y 25│add y 25│add y 25│add y 25│add y 25 │add y 25│add y 25│add y 25│add y 25│add y 25 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │mul y x │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │add y 1 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │mul z y │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │mul y 0 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │add y w │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────┼────────┼────────┼────────┼─────────┤
│add y 7 │add y 15│a

This computes

```python
for i in range(len(s)):
  x = (z%26) + A[i]
  z //= B[i]
  if x != s[i]:
    z = z*26 + (s[i] + C[i])
```


In [88]:
collect_arg2←{{⍎⊃⍵}¨3⌷¨⍵⌷(⍉↑({('inp' (,'w'))≡⍵}¨data24)⊂data24)}

In [89]:
⎕←A←collect_arg2 6

12 11 12 ¯3 10 ¯9 10 ¯7 ¯11 ¯4 14 11 ¯8 ¯10

In [90]:
⎕←B←collect_arg2 5

1 1 1 26 1 26 1 26 26 26 1 1 26 26

In [91]:
⎕←C←collect_arg2 16

7 15 2 15 14 2 15 1 15 15 12 2 13 13

Note that when A≥10 we must have `x != s[i]`, so z must
increase on those steps. Also the steps where A<10 correspond
with the steps where B=26, and each of these steps must avoid
increasing z to have any hope of reaching 0.

Let's just start unrolling this manually, using python indexing
since that's the pseudocode.

```python
# iteration 1: A[0]=12 so z must increase
z = s[0]+7
# iteration 2: A[1]=11 so z must increase
z = 26*(s[0]+7) + (s[1]+15)
# iteration 3: A[2]=12 so z must increase
z = 26*(26*(s[0]+7) + (s[1]+15)) + (s[2]+2)
# iteration 4: A[3]=-3, must have x == s[3]
z = 26*(s[0]+7) + (s[1]+15) # z //= 26
if s[3] != s[2]-1: pass # ...
# iteration 5: A[4]=10 so z must increase
z = 26*(26*(s[0]+7) + (s[1]+15)) + (s[4]+14)
# iteration 6: A[5]=-9, must have x == s[5]
z = 26*(s[0]+7) + (s[1]+15) # z //= 26
if s[5] != s[4]+5: pass # ...
...
```

Ok, let's use a computer.

In [97]:
]dinput
r←constrain_inputs constants;as;bs;cs;z;n;v;k
as bs cs←constants
z←⍬
r←⍬
:For n :In ⍳14
  :If as[n]>0
    z,←⊂(n(cs[n]))
  :Else
    v k←⊃(¯1↑z)
    r,←⊂(n v(k+as[n]))
    z⊢←¯1↓z
  :EndIf
:EndFor

In [99]:
↑constrain_inputs A B C

4 3 ¯1
 6 5 5
 8 7 8
 9 2 4
10 1 3
13 12 ¯6
14 11 2

So we must have digit 4 = (digit 3) - 1, digit 6 = (digit 5) + 5, etc.
This ought to be easy to satisfy.

In [102]:
]dinput
r←find_inputs;d1;d2;d3;d4;d5;d6;d7;d8;d9;d10;d11;d12;d13;d14
r←⍬
:For d3 :In ⍳9
  d4←d3-1
  :If (d4<1)∨(d4>9)
    :Continue
  :EndIf
  :For d5 :In ⍳9
    d6←d5+5
    :If (d6<1)∨(d6>9)
      :Continue
    :EndIf
    :For d7 :In ⍳9
      d8←d7+8
      :If (d8<1)∨(d8>9)
        :Continue
      :EndIf
      :For d2 :In ⍳9
        d9←d2+4
        :If (d9<1)∨(d9>9)
          :Continue
        :EndIf
        :For d1 :In ⍳9
          d10←d1+3
          :If (d10<1)∨(d10>9)
            :Continue
          :EndIf
          :For d12 :In ⍳9
            d13←d12-6
            :If (d13<1)∨(d13>9)
              :Continue
            :EndIf
            :For d11 :In ⍳9
              d14←d11+2
              :If (d14<1)∨(d14>9)
                :Continue
              :EndIf
              r,←10⊥d1 d2 d3 d4 d5 d6 d7 d8 d9 d10 d11 d12 d13 d14
            :EndFor
          :EndFor
        :EndFor
      :EndFor
    :EndFor
  :EndFor
:EndFor

In [107]:
day24_1←⌈/find_inputs

In [108]:
day24_1

65984919997939

In [115]:
run_program data24 (10 10 10 10 10 10 10 10 10 10 10 10 10 10⊤(day24_1))

9 0 0 0

### Part 2

Now we want the min.

In [116]:
day24_2←⌊/find_inputs

In [117]:
day24_2

11211619541713

In [118]:
run_program data24 (10 10 10 10 10 10 10 10 10 10 10 10 10 10⊤(day24_2))

3 0 0 0

## Day 25

[Puzzle link](https://adventofcode.com/2021/day/25)

Sea cucumbers. Sounds like C cucumbers. Good old C, sigh.

In [120]:
⎕←test25←↑⊃⎕NGET 'day25-test.txt' 1

v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>

In [121]:
data25←↑⊃⎕NGET 'day25-input.txt' 1

We need to check if cells to the "east" and "south" on a toroidal grid
are empty. A cell has '>' in the next generation if it west has '>' in
the current generation and it is empty, or if it has '>' in the current
generation and east is nonempty.

In [152]:
'.>v'[1+((2×('v'=⊢))+(((¯1⌽('>'=⊢))∧('.'=⊢))∨((('>'=⊢))∧(1⌽('.'≠⊢)))))test25]

v...>.>vv>
.vv>.>vv..
>.>>v.>..v
>>v>.>.>v.
v>v.vv.v..
.>>.>.v...
.vv...>>v.
v.v..>>v.v
>...v..v..

In [153]:
swim_east←{'.>v'[1+((2×('v'=⊢))+(((¯1⌽('>'=⊢))∧('.'=⊢))∨((('>'=⊢))∧(1⌽('.'≠⊢)))))⍵]}

Then the cucumbers swim south.

In [156]:
'.v>'[1+((2×('>'=⊢))+(((¯1⊖('v'=⊢))∧('.'=⊢))∨((('v'=⊢))∧(1⊖('.'≠⊢)))))(swim_east test25)]

....>.>v.>
v.v>.>v.v.
>v>>..>v..
>>v>v>.>.v
.>v.v...v.
v>>.>vvv..
..v...>>..
vv...>>vv.
>.v.v..v.v

In [157]:
swim_south←{'.v>'[1+((2×('>'=⊢))+(((¯1⊖('v'=⊢))∧('.'=⊢))∨((('v'=⊢))∧(1⊖('.'≠⊢)))))⍵]}

In [160]:
swim←swim_south∘swim_east

In [163]:
(swim⍣58) test25

..>>v>vv..
..v.>>vv..
..>>v>>vv.
..>>>>>vv.
v......>vv
v>v....>>v
vvv.....>>
>vv......>
.>v.vv.v..

### Part 1

How long til the cucumbers stop moving?

In [165]:
day25_1←{n←0 ⋄ _←({n+←1 ⋄ swim ⍵}⍣≡)⍵ ⋄ n}

In [166]:
day25_1 test25

58

In [167]:
day25_1 data25

560

### Part 2

That's it, there's no more today!

## Conclusion

APL was a weird, sometimes frustrating and occasionally beautiful
language. It fit perfectly with this year's deep sea adventure theme -
I felt like I was studying an alien branch of life with totally
different evolutionary history and constraints to the languages I'm
used to. And when I got stuck, I was truly on my own: maybe there
are expert user communities or something somewhere to help beginners,
but I didn't find them.

Mostly, I used Dyalog APL as a functional or procedural language
with some array-language spice. And when used that way, I'd rather
just have a mainstream language with some array-spice, instead.
With more experience, maybe I'd be able to see my way to more
array-esque solutions. For instance, the other day I found this
page in Dyalog's library documentation about
[parallel graph bfs](https://dfns.dyalog.com/n_bfs.htm), which is
a neat idea.

I'm probably not going to spend $200 for that special keyboard.
But I might go on to study some more modern array languages, and
maybe I can apply some of what I learned in my normal day-to-day!